In [ ]:
# Survivor Fantasy Point Simulation Report

## Simulation Parameters

- **Total team-scenario pairs:** 50,000
- **Average team score:** 406.3
- **Total points across all runs:** 20,317,357

### Important: No Player Replacement

This simulation uses **fixed rosters with no replacement**. Each team selects 7 contestants at the start (pre-merge). When a contestant is voted out, they are **not replaced**—the same 7 players are scored for the entire season. Eliminated players stop earning points (no survival, challenges, or tribal) but can still incur penalties (voted out). There is no merge repick in this simulation.


## How the Simulation Works

This section explains the exact rules of each simulated game so you can verify the simulation is correct.

### Overview

- **Monte Carlo simulation:** Each "run" generates one random season scenario. The same set of team rosters is then scored against that scenario. Total team-scenario pairs = (scenario runs) × (rosters per run) × (strategies).
- **One scenario = one full season:** Boot order, challenge winners, idol finds/plays, etc. are determined once per scenario and stay fixed for all teams.
- **One roster = one team:** Each team has a fixed 7-contestant roster. The same roster is scored against every scenario.

---

### Roster Rules (Team Construction)

| Rule | Implementation |
|------|-----------------|
| **Roster size** | 7 contestants |
| **Tribe constraint** | 2 contestants from each of 3 starting tribes (Tribe A, B, C) |
| **Wild card** | 1 additional contestant from any tribe |
| **No replacement** | When a roster member is voted out, they are NOT replaced. The same 7 players are used for the entire season. |
| **No merge repick** | There is no post-merge roster rebuild. The 7 pre-merge picks are the only picks. |

**Roster strategies tested:** `random`, `challenge_beast` (prioritize high challenge_ability), `idol_hunter` (prioritize high idol_likelihood), `utr` (prioritize high survival_bias), `balanced` (same as random).

---

### Contestant Pool

- **24 contestants** total
- **3 starting tribes:** Tribe A (1–8), Tribe B (9–16), Tribe C (17–24)
- **Each contestant has traits:** `challenge_ability`, `idol_likelihood`, `survival_bias` (0–1 scale). These bias random outcomes.

---

### Season Structure (Dynamic)

| Phase | When | Tribes | Immunity | Reward |
|-------|------|--------|----------|--------|
| Pre-merge | Start until swap | 3 tribes | Team (3) | Team (3) |
| Swap | 16 or 17 players left (50% each) | 2 tribes (random) | Team (2) | Team (2) |
| Merge | 12 or 11 players left (50% each) | 1 tribe | Individual | Individual |
| Finale | 3 remain | — | — | No (FTC only) |

- **Tribe swap:** After 7 or 8 eliminations (50% chance each), remaining players are randomly split into 2 tribes.
- **Full merge:** After 4–6 more eliminations (depending on swap point), all remaining players merge into 1 tribe.
- **Total eliminations:** 21 (24 → 3). Episode count varies by swap/merge timing.

---

### Scenario Generation Rules (Per Season)

#### Boot Order
- **Determined once per scenario** using weighted random: each contestant's `survival_bias` (0–1) is their weight.
- **Order is fixed:** First in boot order = episode 1 vote-out, etc. 21 eliminations total (24 → 3).

#### Team Immunity
- **3 tribes (pre-merge):** 1st = +3, 2nd = +1, 3rd = -1
- **2 tribes (swap phase):** 1st = +3, 2nd = +0

#### Team Reward
- **3 tribes:** 1st = +2, 2nd = +1, 3rd = 0
- **2 tribes:** 1st = +2, 2nd = +0

#### Individual Immunity (Post-merge)
- **One winner per episode.** Weighted random using `challenge_ability`. **Winner gets +5.**

#### Vote Matched
- When someone is voted out, a random **65–95%** of the remaining (non-eliminated) players are designated as having voted for that person (on the "right side" of the vote).
- Each of those players gets +2.

#### Voted Out
- **Base penalty:** -4
- **Pocket multiplier:** If the voted-out player had an idol/advantage in pocket, penalty = -4 × 2^(number of items). 1 item = -8, 2 items = -16.
- **Per-vote penalty:** -1 × (votes received). Votes received: random 4–6 pre-merge, 5–9 post-merge.
- **Total:** -4 × 2^pocket + (-1 × votes)

#### Idol Finds
- **3–6 idols found per season** (random, more variability)
- **Random episodes** (from episodes with tribals)
- **Random finders** (contestants who find them)

#### Idol Plays (Success + Failure)
- **Each idol find** can be played at a later tribal. **~55% success rate** per play.
- **Success:** +7 points. **Failure** (played incorrectly): **-2 points**.
- **Must occur after the find.** Only counted when there is a tribal that episode.

#### Clue Reads
- **1–4 clue-read events per season**
- **1–2 readers per clue** (can be shared)
- **Random episodes, random readers**

#### Advantage Plays (non-idol)
- **0–2 successful plays per season**
- **Random episode, random player**

#### Strategic Player of the Episode (+4)
- **Replaces fake idol trick.** Each tribal where someone is voted out, one person from the `vote_matched` list is designated the **strategic driver** (the person who led the vote).
- **Selection:** Random choice **weighted by `survival_bias`** (higher = more likely to be chosen).
- **Points:** +4 to that player.

#### Voted Out with Idol in Pocket
- **17% chance** when someone is voted out that they had 1+ items in pocket
- **3% chance** (given they have an item) that they had 2 items

#### Quit / Medevac
- **Not implemented** in current simulation (quit probability ~0.02, medevac ~0.05—too rare to appear in 100 runs).

---

### Point Values (from config/scoring.yaml)

| Event | Points |
|-------|--------|
| Survival (pre-merge, swap) | +2 |
| Survival (post-merge) | +3 |
| Team immunity 1st | +3 |
| Team immunity 2nd (3 tribes) | +1 |
| Team immunity 2nd (2 tribes) | +0 |
| Team immunity 3rd/last | -1 |
| Team reward 1st | +2 |
| Team reward 2nd (3 tribes) | +1 |
| Team reward 2nd (2 tribes) | +0 |
| Individual immunity | +5 |
| Vote matched | +2 |
| Voted out (base) | -4 |
| Voted out (per vote) | -1 × votes |
| Voted out (pocket) | base × 2^items |
| Clue read | +1 |
| Advantage play | +4 |
| Idol play (success) | +7 |
| Idol play (failure) | -2 |
| Strategic player of episode | +4 |
| Final tribal (top 3) | +5 |
| Win season | +9 |
| Quit | -10 |

---

### Scoring Logic (How Points Are Assigned to a Team)

For each team roster and each episode:

1. **Only roster members still in the game** earn positive points (survival, challenges, vote matched, advantages). Eliminated roster members earn nothing from that point on.
2. **Voted out:** If a roster member is voted out this episode, the penalty is applied: `-4 × 2^pocket_items + (-1 × votes_received)`.
3. **Survival:** Each roster member in `survived` (attended tribal and was not voted out) gets +2 (pre-merge) or +3 (post-merge). Survival is only awarded when a tribal occurred and someone was voted out.
4. **Challenges:** Each roster member on the winning/losing tribe gets the corresponding team immunity/reward points. Individual immunity winner gets +5.
5. **Vote matched:** Each roster member in the `vote_matched` list gets +2.
6. **Advantages:** Clue readers (+1), idol players (+7), advantage players (+4), fake idol trickers (+4).
7. **Placement:** At finale, roster members in final 3 get +5 each; winner gets +9.

---

### What to Verify

- **Eliminations:** 13 tribals with votes (episodes 1–13). 18 − 13 = 5 remain at episode 14. Final three is randomly chosen from those 5; winner from those 3.
- **Survival:** Pre-merge (1–9): +2 per survivor per tribal. Post-merge (10–13): +3 per survivor. Max pre-merge survival for one player: 9 × 2 = 18. Max post-merge: 4 × 3 = 12.
- **Team challenges:** With 3 tribes, each tribe gets exactly one placement (1st, 2nd, or 3rd) per pre-merge episode. Each roster member gets one team immunity and one team reward result per pre-merge episode.
- **Individual immunity:** Exactly one winner per post-merge episode (10–13).


---

## Every Point Feature: Count, Avg per Team, Total Points, % of Total

| Event Type | Total Occurrences | Avg per Team | Total Points | % of All Points |
|------------|-------------------|--------------|--------------|-----------------|
| Survival (pre-merge, +2 per episode) | 2,127,828 | 42.6 | 4,255,656 | 20.95% |
| Survival (post-merge, +3 per episode) | 788,829 | 15.8 | 2,366,487 | 11.65% |
| Survival (swap phase, +2 per episode) | 945,517 | 18.9 | 1,891,034 | 9.31% |
| Team immunity 1st place (+3) | 1,256,011 | 25.1 | 3,768,033 | 18.55% |
| Team immunity 2nd place, 3-team (+1) | 747,737 | 15.0 | 747,737 | 3.68% |
| Team immunity 2nd place, 2-team (+0) | 512,980 | 10.3 | 0 | 0.00% |
| Team immunity 3rd/last (-1) | 747,105 | 14.9 | -747,105 | -3.68% |
| Team reward 1st place (+2) | 1,254,942 | 25.1 | 2,509,884 | 12.35% |
| Team reward 2nd place, 3-team (+1) | 749,523 | 15.0 | 749,523 | 3.69% |
| Team reward 2nd place, 2-team (+0) | 512,689 | 10.3 | 0 | 0.00% |
| Individual immunity win (+5) | 122,648 | 2.5 | 613,240 | 3.02% |
| Vote matched person voted out (+2) | 2,937,217 | 58.7 | 5,874,434 | 28.91% |
| Voted out (base -4 × 2^pocket + -1 per vote) | 309,232 | 6.2 | -3,408,482 | -16.78% |
| First to read clue (+1) | 53,128 | 1.1 | 53,128 | 0.26% |
| Successful advantage play (+4) | 7,033 | 0.1 | 28,132 | 0.14% |
| Successful immunity idol play (+7) | 10,398 | 0.2 | 72,786 | 0.36% |
| Idol played incorrectly (-2) | 9,265 | 0.2 | -18,530 | -0.09% |
| Strategic player of episode (+4) | 308,466 | 6.2 | 1,233,864 | 6.07% |
| Reach final tribal (+5) | 40,768 | 0.8 | 203,840 | 1.00% |
| Win season (+9) | 13,744 | 0.3 | 123,696 | 0.61% |
| Quit (-10) | 0 | 0.0 | 0 | 0.00% |

---

## Example Team Breakdowns

Three sample team-scenario results showing how points break down by event type.

### Example: Lowest Scoring Team (Total: 94 points)

- **Strategy:** random
- **Scenario ID:** 130

| Event Type | Count | Points |
|------------|-------|--------|
| Survival (pre-merge, +2 per episode) | 26 | 52 |
| Survival (swap phase, +2 per episode) | 2 | 4 |
| Team immunity 1st place (+3) | 10 | 30 |
| Team immunity 2nd place, 3-team (+1) | 10 | 10 |
| Team immunity 2nd place, 2-team (+0) | 4 | 0 |
| Team immunity 3rd/last (-1) | 11 | -11 |
| Team reward 1st place (+2) | 13 | 26 |
| Team reward 2nd place, 3-team (+1) | 9 | 9 |
| Team reward 2nd place, 2-team (+0) | 1 | 0 |
| Vote matched person voted out (+2) | 21 | 42 |
| Voted out (base -4 × 2^pocket + -1 per vote) | 7 | -76 |
| Strategic player of episode (+4) | 2 | 8 |

### Example: Median Scoring Team (Total: 406 points)

- **Strategy:** random
- **Scenario ID:** 312

| Event Type | Count | Points |
|------------|-------|--------|
| Survival (pre-merge, +2 per episode) | 44 | 88 |
| Survival (post-merge, +3 per episode) | 13 | 39 |
| Survival (swap phase, +2 per episode) | 25 | 50 |
| Team immunity 1st place (+3) | 30 | 90 |
| Team immunity 2nd place, 3-team (+1) | 15 | 15 |
| Team immunity 2nd place, 2-team (+0) | 12 | 0 |
| Team immunity 3rd/last (-1) | 15 | -15 |
| Team reward 1st place (+2) | 28 | 56 |
| Team reward 2nd place, 3-team (+1) | 15 | 15 |
| Team reward 2nd place, 2-team (+0) | 15 | 0 |
| Individual immunity win (+5) | 1 | 5 |
| Vote matched person voted out (+2) | 65 | 130 |
| Voted out (base -4 × 2^pocket + -1 per vote) | 7 | -79 |
| Strategic player of episode (+4) | 3 | 12 |

### Example: Highest Scoring Team (Total: 771 points)

- **Strategy:** balanced
- **Scenario ID:** 302

| Event Type | Count | Points |
|------------|-------|--------|
| Survival (pre-merge, +2 per episode) | 56 | 112 |
| Survival (post-merge, +3 per episode) | 39 | 117 |
| Survival (swap phase, +2 per episode) | 34 | 68 |
| Team immunity 1st place (+3) | 35 | 105 |
| Team immunity 2nd place, 3-team (+1) | 20 | 20 |
| Team immunity 2nd place, 2-team (+0) | 16 | 0 |
| Team immunity 3rd/last (-1) | 20 | -20 |
| Team reward 1st place (+2) | 35 | 70 |
| Team reward 2nd place, 3-team (+1) | 19 | 19 |
| Team reward 2nd place, 2-team (+0) | 18 | 0 |
| Individual immunity win (+5) | 7 | 35 |
| Vote matched person voted out (+2) | 101 | 202 |
| Voted out (base -4 × 2^pocket + -1 per vote) | 4 | -48 |
| Successful advantage play (+4) | 1 | 4 |
| Successful immunity idol play (+7) | 1 | 7 |
| Strategic player of episode (+4) | 14 | 56 |
| Reach final tribal (+5) | 3 | 15 |
| Win season (+9) | 1 | 9 |

---

## Category Summary

| Category | Avg Points | % of Total |
|----------|------------|------------|
| survival | 170.3 | 41.9% |
| challenges | 152.8 | 37.6% |
| tribal | 117.5 | 28.9% |
| advantages | 27.8 | 6.8% |
| placement | 6.6 | 1.6% |
| penalties | -68.5 | -16.9% |

## Point Distribution (Percentiles)

| Percentile | Points |
|------------|--------|
| 10th | 273 |
| 25th | 335 |
| 50th (median) | 406 |
| 75th | 478 |
| 90th | 538 |

## Strategy Comparison

| Strategy | Avg Score | Min | Max | Count |
|----------|-----------|-----|-----|-------|
| random | 421.6 | 94 | 771 | 10,000 |
| challenge_beast | 370.8 | 99 | 666 | 10,000 |
| idol_hunter | 441.3 | 187 | 707 | 10,000 |
| utr | 376.4 | 99 | 704 | 10,000 |
| balanced | 421.6 | 94 | 771 | 10,000 |

## Balance Notes

- **Survival:** 41.9% of points. Core floor.
- **Challenges:** 37.6% of points. Team + individual immunity/reward.
- **Penalties:** Avg -68.5 per team. Voted-out penalty creates downside.
- **Advantages:** 6.8% of points. Idol/advantage events are rare by design.

# Survivor Fantasy Point Simulation Report

## Simulation Parameters

- **Total team-scenario pairs:** 50,000
- **Average team score:** 406.3
- **Total points across all runs:** 20,317,357

### Important: No Player Replacement

This simulation uses **fixed rosters with no replacement**. Each team selects 7 contestants at the start (pre-merge). When a contestant is voted out, they are **not replaced**—the same 7 players are scored for the entire season. Eliminated players stop earning points (no survival, challenges, or tribal) but can still incur penalties (voted out). There is no merge repick in this simulation.


## How the Simulation Works

This section explains the exact rules of each simulated game so you can verify the simulation is correct.

### Overview

- **Monte Carlo simulation:** Each "run" generates one random season scenario. The same set of team rosters is then scored against that scenario. Total team-scenario pairs = (scenario runs) × (rosters per run) × (strategies).
- **One scenario = one full season:** Boot order, challenge winners, idol finds/plays, etc. are determined once per scenario and stay fixed for all teams.
- **One roster = one team:** Each team has a fixed 7-contestant roster. The same roster is scored against every scenario.

---

### Roster Rules (Team Construction)

| Rule | Implementation |
|------|-----------------|
| **Roster size** | 7 contestants |
| **Tribe constraint** | 2 contestants from each of 3 starting tribes (Tribe A, B, C) |
| **Wild card** | 1 additional contestant from any tribe |
| **No replacement** | When a roster member is voted out, they are NOT replaced. The same 7 players are used for the entire season. |
| **No merge repick** | There is no post-merge roster rebuild. The 7 pre-merge picks are the only picks. |

**Roster strategies tested:** `random`, `challenge_beast` (prioritize high challenge_ability), `idol_hunter` (prioritize high idol_likelihood), `utr` (prioritize high survival_bias), `balanced` (same as random).

---

### Contestant Pool

- **24 contestants** total
- **3 starting tribes:** Tribe A (1–8), Tribe B (9–16), Tribe C (17–24)
- **Each contestant has traits:** `challenge_ability`, `idol_likelihood`, `survival_bias` (0–1 scale). These bias random outcomes.

---

### Season Structure (Dynamic)

| Phase | When | Tribes | Immunity | Reward |
|-------|------|--------|----------|--------|
| Pre-merge | Start until swap | 3 tribes | Team (3) | Team (3) |
| Swap | 16 or 17 players left (50% each) | 2 tribes (random) | Team (2) | Team (2) |
| Merge | 12 or 11 players left (50% each) | 1 tribe | Individual | Individual |
| Finale | 3 remain | — | — | No (FTC only) |

- **Tribe swap:** After 7 or 8 eliminations (50% chance each), remaining players are randomly split into 2 tribes.
- **Full merge:** After 4–6 more eliminations (depending on swap point), all remaining players merge into 1 tribe.
- **Total eliminations:** 21 (24 → 3). Episode count varies by swap/merge timing.

---

### Scenario Generation Rules (Per Season)

#### Boot Order
- **Determined once per scenario** using weighted random: each contestant's `survival_bias` (0–1) is their weight.
- **Order is fixed:** First in boot order = episode 1 vote-out, etc. 21 eliminations total (24 → 3).

#### Team Immunity
- **3 tribes (pre-merge):** 1st = +3, 2nd = +1, 3rd = -1
- **2 tribes (swap phase):** 1st = +3, 2nd = +0

#### Team Reward
- **3 tribes:** 1st = +2, 2nd = +1, 3rd = 0
- **2 tribes:** 1st = +2, 2nd = +0

#### Individual Immunity (Post-merge)
- **One winner per episode.** Weighted random using `challenge_ability`. **Winner gets +5.**

#### Vote Matched
- When someone is voted out, a random **65–95%** of the remaining (non-eliminated) players are designated as having voted for that person (on the "right side" of the vote).
- Each of those players gets +2.

#### Voted Out
- **Base penalty:** -4
- **Pocket multiplier:** If the voted-out player had an idol/advantage in pocket, penalty = -4 × 2^(number of items). 1 item = -8, 2 items = -16.
- **Per-vote penalty:** -1 × (votes received). Votes received: random 4–6 pre-merge, 5–9 post-merge.
- **Total:** -4 × 2^pocket + (-1 × votes)

#### Idol Finds
- **3–6 idols found per season** (random, more variability)
- **Random episodes** (from episodes with tribals)
- **Random finders** (contestants who find them)

#### Idol Plays (Success + Failure)
- **Each idol find** can be played at a later tribal. **~55% success rate** per play.
- **Success:** +7 points. **Failure** (played incorrectly): **-2 points**.
- **Must occur after the find.** Only counted when there is a tribal that episode.

#### Clue Reads
- **1–4 clue-read events per season**
- **1–2 readers per clue** (can be shared)
- **Random episodes, random readers**

#### Advantage Plays (non-idol)
- **0–2 successful plays per season**
- **Random episode, random player**

#### Strategic Player of the Episode (+4)
- **Replaces fake idol trick.** Each tribal where someone is voted out, one person from the `vote_matched` list is designated the **strategic driver** (the person who led the vote).
- **Selection:** Random choice **weighted by `survival_bias`** (higher = more likely to be chosen).
- **Points:** +4 to that player.

#### Voted Out with Idol in Pocket
- **17% chance** when someone is voted out that they had 1+ items in pocket
- **3% chance** (given they have an item) that they had 2 items

#### Quit / Medevac
- **Not implemented** in current simulation (quit probability ~0.02, medevac ~0.05—too rare to appear in 100 runs).

---

### Point Values (from config/scoring.yaml)

| Event | Points |
|-------|--------|
| Survival (pre-merge, swap) | +2 |
| Survival (post-merge) | +3 |
| Team immunity 1st | +3 |
| Team immunity 2nd (3 tribes) | +1 |
| Team immunity 2nd (2 tribes) | +0 |
| Team immunity 3rd/last | -1 |
| Team reward 1st | +2 |
| Team reward 2nd (3 tribes) | +1 |
| Team reward 2nd (2 tribes) | +0 |
| Individual immunity | +5 |
| Vote matched | +2 |
| Voted out (base) | -4 |
| Voted out (per vote) | -1 × votes |
| Voted out (pocket) | base × 2^items |
| Clue read | +1 |
| Advantage play | +4 |
| Idol play (success) | +7 |
| Idol play (failure) | -2 |
| Strategic player of episode | +4 |
| Final tribal (top 3) | +5 |
| Win season | +9 |
| Quit | -10 |

---

### Scoring Logic (How Points Are Assigned to a Team)

For each team roster and each episode:

1. **Only roster members still in the game** earn positive points (survival, challenges, vote matched, advantages). Eliminated roster members earn nothing from that point on.
2. **Voted out:** If a roster member is voted out this episode, the penalty is applied: `-4 × 2^pocket_items + (-1 × votes_received)`.
3. **Survival:** Each roster member in `survived` (attended tribal and was not voted out) gets +2 (pre-merge) or +3 (post-merge). Survival is only awarded when a tribal occurred and someone was voted out.
4. **Challenges:** Each roster member on the winning/losing tribe gets the corresponding team immunity/reward points. Individual immunity winner gets +5.
5. **Vote matched:** Each roster member in the `vote_matched` list gets +2.
6. **Advantages:** Clue readers (+1), idol players (+7), advantage players (+4), fake idol trickers (+4).
7. **Placement:** At finale, roster members in final 3 get +5 each; winner gets +9.

---

### What to Verify

- **Eliminations:** 13 tribals with votes (episodes 1–13). 18 − 13 = 5 remain at episode 14. Final three is randomly chosen from those 5; winner from those 3.
- **Survival:** Pre-merge (1–9): +2 per survivor per tribal. Post-merge (10–13): +3 per survivor. Max pre-merge survival for one player: 9 × 2 = 18. Max post-merge: 4 × 3 = 12.
- **Team challenges:** With 3 tribes, each tribe gets exactly one placement (1st, 2nd, or 3rd) per pre-merge episode. Each roster member gets one team immunity and one team reward result per pre-merge episode.
- **Individual immunity:** Exactly one winner per post-merge episode (10–13).


---

## Every Point Feature: Count, Avg per Team, Total Points, % of Total

| Event Type | Total Occurrences | Avg per Team | Total Points | % of All Points |
|------------|-------------------|--------------|--------------|-----------------|
| Survival (pre-merge, +2 per episode) | 2,127,828 | 42.6 | 4,255,656 | 20.95% |
| Survival (post-merge, +3 per episode) | 788,829 | 15.8 | 2,366,487 | 11.65% |
| Survival (swap phase, +2 per episode) | 945,517 | 18.9 | 1,891,034 | 9.31% |
| Team immunity 1st place (+3) | 1,256,011 | 25.1 | 3,768,033 | 18.55% |
| Team immunity 2nd place, 3-team (+1) | 747,737 | 15.0 | 747,737 | 3.68% |
| Team immunity 2nd place, 2-team (+0) | 512,980 | 10.3 | 0 | 0.00% |
| Team immunity 3rd/last (-1) | 747,105 | 14.9 | -747,105 | -3.68% |
| Team reward 1st place (+2) | 1,254,942 | 25.1 | 2,509,884 | 12.35% |
| Team reward 2nd place, 3-team (+1) | 749,523 | 15.0 | 749,523 | 3.69% |
| Team reward 2nd place, 2-team (+0) | 512,689 | 10.3 | 0 | 0.00% |
| Individual immunity win (+5) | 122,648 | 2.5 | 613,240 | 3.02% |
| Vote matched person voted out (+2) | 2,937,217 | 58.7 | 5,874,434 | 28.91% |
| Voted out (base -4 × 2^pocket + -1 per vote) | 309,232 | 6.2 | -3,408,482 | -16.78% |
| First to read clue (+1) | 53,128 | 1.1 | 53,128 | 0.26% |
| Successful advantage play (+4) | 7,033 | 0.1 | 28,132 | 0.14% |
| Successful immunity idol play (+7) | 10,398 | 0.2 | 72,786 | 0.36% |
| Idol played incorrectly (-2) | 9,265 | 0.2 | -18,530 | -0.09% |
| Strategic player of episode (+4) | 308,466 | 6.2 | 1,233,864 | 6.07% |
| Reach final tribal (+5) | 40,768 | 0.8 | 203,840 | 1.00% |
| Win season (+9) | 13,744 | 0.3 | 123,696 | 0.61% |
| Quit (-10) | 0 | 0.0 | 0 | 0.00% |

---

## Example Team Breakdowns

Three sample team-scenario results showing how points break down by event type.

### Example: Lowest Scoring Team (Total: 94 points)

- **Strategy:** random
- **Scenario ID:** 130

| Event Type | Count | Points |
|------------|-------|--------|
| Survival (pre-merge, +2 per episode) | 26 | 52 |
| Survival (swap phase, +2 per episode) | 2 | 4 |
| Team immunity 1st place (+3) | 10 | 30 |
| Team immunity 2nd place, 3-team (+1) | 10 | 10 |
| Team immunity 2nd place, 2-team (+0) | 4 | 0 |
| Team immunity 3rd/last (-1) | 11 | -11 |
| Team reward 1st place (+2) | 13 | 26 |
| Team reward 2nd place, 3-team (+1) | 9 | 9 |
| Team reward 2nd place, 2-team (+0) | 1 | 0 |
| Vote matched person voted out (+2) | 21 | 42 |
| Voted out (base -4 × 2^pocket + -1 per vote) | 7 | -76 |
| Strategic player of episode (+4) | 2 | 8 |

### Example: Median Scoring Team (Total: 406 points)

- **Strategy:** random
- **Scenario ID:** 312

| Event Type | Count | Points |
|------------|-------|--------|
| Survival (pre-merge, +2 per episode) | 44 | 88 |
| Survival (post-merge, +3 per episode) | 13 | 39 |
| Survival (swap phase, +2 per episode) | 25 | 50 |
| Team immunity 1st place (+3) | 30 | 90 |
| Team immunity 2nd place, 3-team (+1) | 15 | 15 |
| Team immunity 2nd place, 2-team (+0) | 12 | 0 |
| Team immunity 3rd/last (-1) | 15 | -15 |
| Team reward 1st place (+2) | 28 | 56 |
| Team reward 2nd place, 3-team (+1) | 15 | 15 |
| Team reward 2nd place, 2-team (+0) | 15 | 0 |
| Individual immunity win (+5) | 1 | 5 |
| Vote matched person voted out (+2) | 65 | 130 |
| Voted out (base -4 × 2^pocket + -1 per vote) | 7 | -79 |
| Strategic player of episode (+4) | 3 | 12 |

### Example: Highest Scoring Team (Total: 771 points)

- **Strategy:** balanced
- **Scenario ID:** 302

| Event Type | Count | Points |
|------------|-------|--------|
| Survival (pre-merge, +2 per episode) | 56 | 112 |
| Survival (post-merge, +3 per episode) | 39 | 117 |
| Survival (swap phase, +2 per episode) | 34 | 68 |
| Team immunity 1st place (+3) | 35 | 105 |
| Team immunity 2nd place, 3-team (+1) | 20 | 20 |
| Team immunity 2nd place, 2-team (+0) | 16 | 0 |
| Team immunity 3rd/last (-1) | 20 | -20 |
| Team reward 1st place (+2) | 35 | 70 |
| Team reward 2nd place, 3-team (+1) | 19 | 19 |
| Team reward 2nd place, 2-team (+0) | 18 | 0 |
| Individual immunity win (+5) | 7 | 35 |
| Vote matched person voted out (+2) | 101 | 202 |
| Voted out (base -4 × 2^pocket + -1 per vote) | 4 | -48 |
| Successful advantage play (+4) | 1 | 4 |
| Successful immunity idol play (+7) | 1 | 7 |
| Strategic player of episode (+4) | 14 | 56 |
| Reach final tribal (+5) | 3 | 15 |
| Win season (+9) | 1 | 9 |

---

## Category Summary

| Category | Avg Points | % of Total |
|----------|------------|------------|
| survival | 170.3 | 41.9% |
| challenges | 152.8 | 37.6% |
| tribal | 117.5 | 28.9% |
| advantages | 27.8 | 6.8% |
| placement | 6.6 | 1.6% |
| penalties | -68.5 | -16.9% |

## Point Distribution (Percentiles)

| Percentile | Points |
|------------|--------|
| 10th | 273 |
| 25th | 335 |
| 50th (median) | 406 |
| 75th | 478 |
| 90th | 538 |

## Strategy Comparison

| Strategy | Avg Score | Min | Max | Count |
|----------|-----------|-----|-----|-------|
| random | 421.6 | 94 | 771 | 10,000 |
| challenge_beast | 370.8 | 99 | 666 | 10,000 |
| idol_hunter | 441.3 | 187 | 707 | 10,000 |
| utr | 376.4 | 99 | 704 | 10,000 |
| balanced | 421.6 | 94 | 771 | 10,000 |

## Balance Notes

- **Survival:** 41.9% of points. Core floor.
- **Challenges:** 37.6% of points. Team + individual immunity/reward.
- **Penalties:** Avg -68.5 per team. Voted-out penalty creates downside.
- **Advantages:** 6.8% of points. Idol/advantage events are rare by design.

# Phase 1: Pricing Simulation Report

## Summary

- **Budget:** $1,000,000
- **Roster rules:** Min 1 per tribe, max 7 players, no minimum roster size (5–7 valid)
- **Price estimation runs:** 800
- **Scenario runs (scoring):** 300
- **Unique roster compositions:** 29

---

## Price Distribution

| Metric | Value |
|--------|-------|
| Min price | $80,000 |
| Max price | $225,000 |
| Sum (all 24) | $3,000,000 |
| Top 5 by expected (cost) | $950,000 |
| Top 6 by expected (cost) | $1,095,000 |
| Top 7 by expected (cost) | $1,235,000 |

**Viability:** Top 5 ≈ budget (5 premium viable). Top 6–7 progressively more expensive (6 premium or 7 mid-tier).

---

## Contestant Prices & Expected Points

| Contestant | Tribe | Expected Pts | Price |
|------------|-------|--------------|-------|
| c21 | Tribe C | 70.6 | $225,000 |
| c05 | Tribe A | 68.3 | $195,000 |
| c13 | Tribe B | 67.3 | $185,000 |
| c23 | Tribe C | 66.7 | $175,000 |
| c07 | Tribe A | 66.2 | $170,000 |
| c15 | Tribe B | 63.7 | $145,000 |
| c11 | Tribe B | 63.1 | $140,000 |
| c01 | Tribe A | 63.1 | $140,000 |
| c09 | Tribe B | 62.7 | $135,000 |
| c17 | Tribe C | 62.4 | $135,000 |
| c03 | Tribe A | 60.8 | $120,000 |
| c18 | Tribe C | 60.6 | $120,000 |
| c08 | Tribe A | 60.6 | $120,000 |
| c10 | Tribe B | 60.0 | $115,000 |
| c19 | Tribe C | 59.2 | $110,000 |
| c24 | Tribe C | 57.8 | $100,000 |
| c02 | Tribe A | 57.5 | $95,000 |
| c04 | Tribe A | 56.1 | $90,000 |
| c16 | Tribe B | 55.8 | $85,000 |
| c20 | Tribe C | 54.3 | $80,000 |
| c12 | Tribe B | 53.9 | $80,000 |
| c06 | Tribe A | 50.6 | $80,000 |
| c14 | Tribe B | 49.4 | $80,000 |
| c22 | Tribe C | 49.3 | $80,000 |

---

## Strategy Descriptions

| Strategy | Description |
|----------|-------------|
| **max_expected** | Picks up to 7 highest expected-point players under budget (min 1 per tribe). |
| **value** | Picks players with best points-per-dollar. Favors cheaper players; typically 7 mid-tier. |
| **balanced** | Value (pts/price) with tiebreaker toward higher expected. |
| **mid_tier** | Only considers middle 50% of expected points. Avoids expensive stars and weak scrubs. |
| **stars_and_scrubs** | Picks 2 expensive stars, then fills with cheapest (min 1 per tribe). |
| **five_premium** | Exactly 5 premium players (highest expected, min 1 per tribe). Tests 5-stars viability. |
| **six_premium** | Exactly 6 premium players (min 1 per tribe). Tests 6-stars vs 7-mid tradeoff. |
| **random** | Randomly selects valid rosters under budget. |

---

## Strategy Performance (Budget-Constrained Rosters)

| Strategy | Avg Score | Avg Cost | Min | Max | Count |
|----------|-----------|----------|-----|-----|-------|
| max_expected | 341.1 | $950,000 | 127 | 547 | 7500 |
| value | 370.8 | $575,000 | 144 | 634 | 7500 |
| balanced | 341.1 | $950,000 | 127 | 547 | 7500 |
| mid_tier | 407.2 | $750,000 | 195 | 703 | 7500 |
| stars_and_scrubs | 397.8 | $820,000 | 167 | 736 | 7500 |
| five_premium | 341.1 | $950,000 | 127 | 547 | 7500 |
| six_premium | 341.1 | $950,000 | 127 | 547 | 7500 |
| random | 411.1 | $843,400 | 107 | 756 | 7500 |

---

## Example Rosters & Top Picks by Strategy

### max_expected

- **Example roster:** c05, c07, c13, c21, c23 (5 players)
- **Cost:** $950,000 | **Expected pts (sum):** 339.1
- **Most picked:** c05 (100%), c13 (100%), c21 (100%), c23 (100%), c07 (100%)

### value

- **Example roster:** c04, c06, c12, c14, c16, c20, c22 (7 players)
- **Cost:** $575,000 | **Expected pts (sum):** 369.4
- **Most picked:** c06 (100%), c12 (100%), c20 (100%), c16 (100%), c04 (100%)

### balanced

- **Example roster:** c05, c07, c13, c21, c23 (5 players)
- **Cost:** $950,000 | **Expected pts (sum):** 339.1
- **Most picked:** c05 (100%), c13 (100%), c21 (100%), c23 (100%), c07 (100%)

### mid_tier

- **Example roster:** c02, c03, c04, c10, c18, c19, c24 (7 players)
- **Cost:** $750,000 | **Expected pts (sum):** 412.1
- **Most picked:** c04 (100%), c10 (100%), c24 (100%), c02 (100%), c19 (100%)

### stars_and_scrubs

- **Example roster:** c05, c06, c12, c14, c20, c21, c22 (7 players)
- **Cost:** $820,000 | **Expected pts (sum):** 396.4
- **Most picked:** c21 (100%), c05 (100%), c12 (100%), c06 (100%), c14 (100%)

### five_premium

- **Example roster:** c05, c07, c13, c21, c23 (5 players)
- **Cost:** $950,000 | **Expected pts (sum):** 339.1
- **Most picked:** c05 (100%), c13 (100%), c21 (100%), c23 (100%), c07 (100%)

### six_premium

- **Example roster:** c05, c07, c13, c21, c23 (5 players)
- **Cost:** $950,000 | **Expected pts (sum):** 339.1
- **Most picked:** c05 (100%), c13 (100%), c21 (100%), c23 (100%), c07 (100%)

### random

- **Example roster:** c07, c10, c14, c15, c16, c19, c22 (7 players)
- **Cost:** $785,000 | **Expected pts (sum):** 403.6
- **Most picked:** c14 (44%), c02 (44%), c22 (40%), c18 (40%), c10 (36%)

---

## Findings

1. **Price spread:** Wider variation ($40K–$230K in $5K increments). Top players much more expensive; bottom cheaper.
2. **Roster diversity:** Unique roster count indicates how many distinct team compositions the pricing allows.
3. **No dominant strategy:** mid_tier, random, stars_and_scrubs, and six_premium all competitive; no single clear winner.
4. **5–6 premium viable:** Top 5 fits under budget; 5 or 6 premium players is a real alternative to 7 mid-tier.
5. **Roster flexibility:** Min 1 per tribe, max 7, no minimum—enables 5-stars, 6-stars, or 7-mid strategies.


# Pricing System Improvement Report

**Date:** February 19, 2025  
**Purpose:** Document changes made to increase unique roster combinations and improve roster diversity.

---

## Summary of Changes

Three improvements were implemented to increase the number of viable, distinct roster combinations:

| Change | Before | After |
|--------|--------|-------|
| **target_top5_sum** | $950,000 | $900,000 |
| **price_increment** | $5,000 | $2,500 |
| **Tie-breaking** | Deterministic (first match) | Random among ties |

---

## 1. Pricing Configuration (`config/pricing.yaml`)

### target_top5_sum: $950,000 → $900,000

**Rationale:** With top 5 at exactly budget, there was no slack for tradeoffs. Lowering to $900K creates ~$100K of headroom, enabling:

- **6-player premium rosters** — A 6th mid-tier player can now fit under budget alongside 5 premium picks
- **Swaps within premium tier** — e.g., swap one $170K player for a $145K player and add a value pick
- **More viable combinations** — Multiple roster compositions can now hit the budget

### price_increment: $5,000 → $2,500

**Rationale:** Finer granularity creates more distinct price points (37 vs 37 in range, but more precise rounding). This allows:

- More precise budget allocation
- Fewer players clustered at identical prices
- More meaningful tradeoff decisions between similar players

---

## 2. Roster Generator Tie-Breaking (`src/roster_generator.py`)

When multiple players had identical scores (expected points, value ratio, etc.), the generator always picked the first in sorted order. This made strategies like `max_expected`, `value`, `balanced`, and `mid_tier` fully deterministic.

**Changes:**

- **`generate_budget_roster`:** When picking by score, if multiple candidates tie for best score, choose randomly among them
- **`_generate_premium_roster`:** When picking best by expected points (per tribe or fill slots), break ties randomly
- **`_generate_stars_and_scrubs`:** When picking 2 stars by expected points, break ties randomly

**Effect:** Strategies that previously produced the same roster every run can now produce variation when ties occur.

---

## 3. Simulation Parameters

- **Rosters per strategy:** Increased from 15 to 25 (in `run_pricing_simulation.py`) to better exercise tie-breaking and capture roster diversity
- **Price estimation runs:** 2,000  
- **Scenario runs:** 300  

---

## Before vs After Comparison

| Metric | Before | After |
|--------|--------|-------|
| **Unique roster compositions** | 29 | 30 |
| **Top 5 by expected (cost)** | $950,000 | ~$902,500 |
| **Top 6 by expected (cost)** | $1,095,000 | ~$1,047,500 |
| **Price increments** | $5,000 | $2,500 |
| **6-premium viable?** | No (fell back to 5) | Yes |

### Strategy Behavior Changes

| Strategy | Before | After |
|----------|--------|-------|
| **max_expected** | 5 players (c05, c07, c13, c21, c23), $950K | 6 players (c05, c11, c13, c20, c21, c23), ~$987K |
| **six_premium** | 5 players (same as five_premium) | 6 players (c05, c11, c13, c20, c21, c23) |
| **five_premium** | 5 players | 5 players (c05, c11, c13, c21, c23) |

**Key improvement:** The `six_premium` strategy now successfully builds 6-player premium rosters instead of falling back to 5, validating that 6 premium players can fit under budget with the new pricing.

---

## New Simulation Results

See `pricing_report.md` and `pricing_analysis.json` in this directory for the full output of the latest run.

### Highlights

- **Unique rosters:** 30 (slight increase; more variety possible with additional rosters per strategy)
- **6-premium rosters:** Now viable — max_expected, balanced, and six_premium all build 6-player rosters
- **Price spread:** $80,000–$207,500 in $2,500 increments
- **Strategy performance:** mid_tier (413.3 avg), random (411.0), stars_and_scrubs (403.7) remain competitive

---

## Recommendations for Further Improvement

If more roster diversity is desired:

1. **Increase rosters per strategy** — e.g., `--rosters 40` or higher
2. **Flatten price curve** — Try `price_curve: 1.4` to compress the spread and create more mid-tier tradeoffs
3. **Lower target_top5_sum further** — e.g., $850K to allow 6 premium players to fit more comfortably
4. **Widen price range** — Lower `price_min` or raise `price_max` for more distinct tiers

---

## Files Modified

- `scripts/point_simulation/config/pricing.yaml` — target_top5_sum, price_increment
- `scripts/point_simulation/src/roster_generator.py` — tie-breaking in generate_budget_roster, _generate_premium_roster, _generate_stars_and_scrubs
- `scripts/point_simulation/run_pricing_simulation.py` — default rosters per strategy (15→25), dynamic price increment in report


# Roster Diversity & Total Team Options Report

**Date:** February 19, 2025  
**Simulation:** Phase 1 Pricing with Sampled Rosters

---

## Executive Summary

This report documents the **total number of valid team options** under the current pricing system and the results of testing a diverse sample of rosters across Monte Carlo scenarios.

| Metric | Value |
|--------|-------|
| **Total valid team options** | 426,187 |
| **Rosters tested in simulation** | 500 |
| **Scenarios per roster** | 300 |
| **Total simulation runs** | 150,000 |

---

## Total Valid Team Options

Given the pricing system constraints:

- **Budget:** $1,000,000
- **Roster size:** 5, 6, or 7 players
- **Tribe rule:** Minimum 1 player from each of 3 tribes
- **24 contestants** (8 per tribe)

The system supports **426,187 distinct valid roster combinations**:

| Roster Size | Valid Combinations |
|-------------|---------------------|
| 5 players | 29,568 |
| 6 players | 110,552 |
| 7 players | 286,067 |
| **Total** | **426,187** |

7-player rosters dominate (67% of valid options) because they allow more budget flexibility and more ways to satisfy the tribe constraint.

---

## Simulation Approach

### Previous Limitation

The original simulation used 8 strategy-based roster generators. Most strategies were deterministic, producing the same roster every run. Only ~30 unique rosters were tested across 200 generated entries.

### New Approach: Uniform Sampling

1. **Enumerate** all valid roster combinations (min 1 per tribe, under budget)
2. **Sample** N rosters uniformly from this set (rejection sampling with deduplication)
3. **Score** each sampled roster across 300 Monte Carlo season scenarios
4. **Report** total valid options, rosters tested, and score distribution

### Implementation

- **`src/roster_enumerator.py`**: `count_valid_rosters()`, `sample_valid_rosters()`
- **CLI:** `python run_pricing_simulation.py --sample-rosters 500`

---

## Simulation Results (500 Sampled Rosters)

| Metric | Value |
|--------|-------|
| **Avg score (sampled)** | 363.5 |
| **Min score** | 21 |
| **Max score** | 801 |
| **Avg cost** | $761,775 |
| **Unique rosters tested** | 500 |

### Score Distribution

- Sampled rosters span a wide range (21–801 points), reflecting the diversity of valid team compositions
- Average cost ($761K) is below budget, as many valid rosters use mid-tier players
- Top performers likely combine premium players with value picks under the tribe constraint

---

## Key Findings

1. **Pricing supports substantial diversity:** 426K+ valid team options give players meaningful choice without overwhelming complexity.

2. **7-player rosters are most common:** 286K of 426K options (67%) use all 7 roster slots, enabling deeper rosters with value picks.

3. **Sampling enables broad testing:** Testing 500 uniformly sampled rosters provides a representative view of the strategy space vs. the previous ~30 strategy-derived rosters.

4. **Score variance is high:** Min 21 to max 801 points across scenarios indicates roster composition and scenario outcomes both matter significantly.

---

## Files & Commands

| File | Description |
|------|-------------|
| `pricing_report.md` | Full simulation report (prices, performance, examples) |
| `pricing_analysis.json` | Raw analysis data |
| `scripts/point_simulation/src/roster_enumerator.py` | Roster counting and sampling logic |

**Run simulation with sampling:**
```bash
python run_pricing_simulation.py --sample-rosters 500 --output ../../output/simulation
```

**Run with more rosters:**
```bash
python run_pricing_simulation.py --sample-rosters 1000 --output ../../output/simulation
```


# Phase 1: Pricing Simulation Report

## Summary

- **Budget:** $1,000,000
- **Roster rules:** Exactly 7 players required, min 1 per tribe
- **Price estimation runs:** 2000
- **Scenario runs (scoring):** 300
- **Unique roster compositions tested:** 500

### Total Valid Team Options (Pricing System)

- **Total valid roster combinations:** 311,808
- 7-player rosters: 311,808
- **Rosters sampled for simulation:** 500

---


## Price Distribution

| Metric | Value |
|--------|-------|
| Min price | $80,000 |
| Max price | $167,500 |
| Sum (all 24) | $2,562,500 |
| Top 5 by expected (cost) | $742,500 |
| Top 6 by expected (cost) | $862,500 |
| Top 7 by expected (cost) | $980,000 |

**Viability:** Top 7 by expected points ≈ budget (7-player rosters fit under budget).

---

## Players by Price (Breakdown by Price Tier)

| Price | Contestants |
|-------|-------------|
| $167,500 | c05, c21 |
| $152,500 | c13 |
| $132,500 | c23 |
| $122,500 | c11 |
| $120,000 | c17 |
| $117,500 | c07 |
| $115,000 | c19, c01 |
| $112,500 | c15 |
| $110,000 | c09, c03 |
| $95,000 | c08 |
| $92,500 | c10, c18, c24 |
| $85,000 | c16 |
| $82,500 | c02 |
| $80,000 | c04, c20, c12, c06, c22, c14 |

---

## Point Breakdown (Aggregate Across All Runs)

| Category | Total Points | % of Total |
|----------|--------------|------------|
| survival | 26,406,518 | 41.9% |
| challenges | 23,422,414 | 37.2% |
| tribal | 18,179,452 | 28.9% |
| advantages | 4,148,470 | 6.6% |
| placement | 1,049,051 | 1.7% |
| penalties | -10,194,339 | -16.2% |

---

## Event Breakdown (Points by Event Type)

| Event Type | Total Points | % of Total | Count |
|------------|--------------|------------|-------|
| Vote matched | 18,179,452 | 28.9% | 9,089,726 |
| Survival (pre-merge) | 12,945,188 | 20.5% | 6,472,594 |
| Team immunity 1st | 11,577,864 | 18.4% | 3,859,288 |
| team_reward_first | 7,717,554 | 12.2% | 3,858,777 |
| Survival (post-merge) | 7,547,514 | 12.0% | 2,515,838 |
| Survival (swap) | 5,913,816 | 9.4% | 2,956,908 |
| strategic_player | 3,676,308 | 5.8% | 919,077 |
| team_immunity_second_three | 2,271,313 | 3.6% | 2,271,313 |
| team_reward_second_three | 2,269,281 | 3.6% | 2,269,281 |
| Individual immunity | 1,848,200 | 2.9% | 369,640 |
| Final tribal | 655,670 | 1.0% | 131,134 |
| Win season | 393,381 | 0.6% | 43,709 |
| idol_play | 221,102 | 0.4% | 31,586 |
| clue_read | 163,624 | 0.3% | 163,624 |
| advantage_play | 87,436 | 0.1% | 21,859 |
| team_immunity_second_two | 0 | 0.0% | 1,584,234 |
| team_reward_second_two | 0 | 0.0% | 1,586,265 |
| idol_failure | -58,640 | -0.1% | 29,320 |
| team_immunity_last | -2,261,798 | -3.6% | 2,261,798 |
| Voted out | -10,135,699 | -16.1% | 918,866 |

---

## Contestant Prices & Expected Points

| Contestant | Tribe | Expected Pts | Price |
|------------|-------|--------------|-------|
| c05 | Tribe A | 69.0 | $167,500 |
| c21 | Tribe C | 69.0 | $167,500 |
| c13 | Tribe B | 67.3 | $152,500 |
| c23 | Tribe C | 65.0 | $132,500 |
| c11 | Tribe B | 64.0 | $122,500 |
| c17 | Tribe C | 63.5 | $120,000 |
| c07 | Tribe A | 63.3 | $117,500 |
| c19 | Tribe C | 63.0 | $115,000 |
| c01 | Tribe A | 62.9 | $115,000 |
| c15 | Tribe B | 62.7 | $112,500 |
| c09 | Tribe B | 62.2 | $110,000 |
| c03 | Tribe A | 62.1 | $110,000 |
| c08 | Tribe A | 60.0 | $95,000 |
| c10 | Tribe B | 59.6 | $92,500 |
| c18 | Tribe C | 59.5 | $92,500 |
| c24 | Tribe C | 59.4 | $92,500 |
| c16 | Tribe B | 58.1 | $85,000 |
| c02 | Tribe A | 57.9 | $82,500 |
| c04 | Tribe A | 54.8 | $80,000 |
| c20 | Tribe C | 54.7 | $80,000 |
| c12 | Tribe B | 53.9 | $80,000 |
| c06 | Tribe A | 51.7 | $80,000 |
| c22 | Tribe C | 49.5 | $80,000 |
| c14 | Tribe B | 47.6 | $80,000 |

---

## Strategy Descriptions

| Strategy | Description |
|----------|-------------|
| **sampled** | Uniformly sampled from all valid roster combinations (min 1 per tribe, under budget). |
| **max_expected** | Picks up to 7 highest expected-point players under budget (min 1 per tribe). |
| **value** | Picks players with best points-per-dollar. Favors cheaper players; typically 7 mid-tier. |
| **balanced** | Value (pts/price) with tiebreaker toward higher expected. |
| **mid_tier** | Only considers middle 50% of expected points. Avoids expensive stars and weak scrubs. |
| **stars_and_scrubs** | Picks 2 expensive stars, then fills with cheapest (min 1 per tribe). |
| **five_premium** | Exactly 5 premium players (highest expected, min 1 per tribe). Tests 5-stars viability. |
| **six_premium** | Exactly 6 premium players (min 1 per tribe). Tests 6-stars vs 7-mid tradeoff. |
| **random** | Randomly selects valid rosters under budget. |

---

## Strategy Performance (Budget-Constrained Rosters)

| Strategy | Avg Score | Avg Cost | Min | Max | Count |
|----------|-----------|----------|-----|-----|-------|
| sampled | 420.1 | $749,145 | 66 | 801 | 150000 |

---

## Example Rosters & Top Picks by Strategy

### sampled

- **Example roster:** c01, c04, c05, c08, c09, c20, c21 (7 players)
- **Cost:** $815,000 | **Expected pts (sum):** 432.5
- **Most picked:** c05 (34%), c03 (34%), c08 (32%), c24 (32%), c22 (31%)

---

## Findings

1. **Price spread:** $80,000–$167,500 in $2,500 increments.
2. **Total valid team options:** 311,808 distinct roster combinations under budget (min 1 per tribe, 7–7 players).
3. **Rosters tested:** 500 unique compositions scored across 300 scenarios.
4. **7 players required:** All rosters must have exactly 7 players; top 7 by expected points ≈ budget.
5. **Roster flexibility:** Min 1 per tribe, 7 players—enables diverse team compositions within budget.


# Dynamic Pricing Simulation Report

## Overview

This report documents how **episode-reactive price changes** work and whether they create **multiple viable replacement options** when a player gets voted off.

**Goal:** When someone is voted off and teams need to replace them, there should NOT be one obvious player everyone wants. Multiple options at different price points should be viable.

---

## How Price Changes Work

### Update Formula

After each episode:

1. **Points earned** — Each contestant gets points for: survival, immunity, vote matched, etc.
2. **Relative performance** — Compare each player's points to the episode average.
3. **Price delta** — `(points - avg) / range × reactivity`
   - Strong performers (above avg): price increases
   - Weak performers (below avg): price decreases
4. **Diversity compression** — Prices are pulled toward the median to create clusters.
   - Reduces one player becoming dominant; creates value bands.

### Key Parameters

| Parameter | Value | Effect |
|-----------|-------|--------|
| price_reactivity | 0.15 | Max % price move per episode |
| diversity_compression | 0.3 | Pull toward median for clustering |
| replacement_value_tolerance | 0.1 | Viable = within 10% of best value |

---

## Replacement Diversity Results

When a player is voted off, we count how many replacement options are **viable** (within 10% of best value, affordable with freed budget).

| Metric | Value |
|--------|-------|
| Replacement events simulated | 1,050 |
| Avg viable options per event | 4.2 |
| Min viable options (worst case) | 0 |
| % of events with 3+ viable options | 57.1% |

---

## Example Replacement Scenarios

### Event 1: c07 voted off (Episode 1)

- **Viable replacements:** 10
- **Affordable options:** 20
- **Viable picks:** c16, c06, c12, c02, c20, c24, c04, c14...

### Event 2: c06 voted off (Episode 2)

- **Viable replacements:** 0
- **Affordable options:** 0

### Event 3: c18 voted off (Episode 3)

- **Viable replacements:** 7
- **Affordable options:** 9
- **Viable picks:** c16, c12, c02, c20, c24, c04, c19

### Event 4: c17 voted off (Episode 4)

- **Viable replacements:** 11
- **Affordable options:** 13
- **Viable picks:** c15, c16, c12, c03, c02, c08, c20, c10...

### Event 5: c22 voted off (Episode 5)

- **Viable replacements:** 2
- **Affordable options:** 2
- **Viable picks:** c20, c14

### Event 6: c03 voted off (Episode 6)

- **Viable replacements:** 7
- **Affordable options:** 10
- **Viable picks:** c16, c02, c08, c10, c24, c04, c19

### Event 7: c11 voted off (Episode 7)

- **Viable replacements:** 9
- **Affordable options:** 16
- **Viable picks:** c23, c15, c01, c13, c08, c10, c09, c05...

### Event 8: c01 voted off (Episode 8)

- **Viable replacements:** 6
- **Affordable options:** 15
- **Viable picks:** c23, c15, c13, c10, c09, c05

### Event 9: c13 voted off (Episode 9)

- **Viable replacements:** 5
- **Affordable options:** 14
- **Viable picks:** c23, c15, c10, c09, c05

### Event 10: c16 voted off (Episode 10)

- **Viable replacements:** 4
- **Affordable options:** 9
- **Viable picks:** c15, c10, c09, c05

---

## Price Evolution (Sample Scenario)

**After Episode 0:** c01=$127,500, c02=$90,000, c03=$107,500, c04=$95,000, c05=$180,000
**After Episode 1:** c01=$127,500, c02=$100,000, c03=$112,500, c04=$105,000, c05=$162,500
**After Episode 2:** c01=$130,000, c02=$107,500, c03=$115,000, c04=$110,000, c05=$150,000
**After Episode 3:** c01=$130,000, c02=$112,500, c03=$117,500, c04=$115,000, c05=$142,500
**After Episode 4:** c01=$127,500, c02=$115,000, c03=$120,000, c04=$117,500, c05=$137,500

---

## Price Change Formula Example

When a player wins individual immunity (+5 pts) and another gets vote matched (+2 pts) while the tribe loses immunity (-1):

- **Immunity winner:** pts = 5, avg ≈ 1.5 → delta_norm positive → price goes **up**
- **Vote matched:** pts = 2, avg ≈ 1.5 → delta_norm slightly positive → price goes **up** slightly
- **Tribe lost immunity:** pts = -1, avg ≈ 1.5 → delta_norm negative → price goes **down**
- **Voted out:** pts = -8 (e.g.), avg ≈ 1.5 → delta_norm very negative → price drops (but they're removed from pool)

Diversity compression then pulls all prices toward the median, so similar performers cluster at similar prices.

---

## Edge Cases

- **0 affordable:** When a cheap player is voted off, budget freed is low. If all remaining players cost more, no one is affordable. Consider: minimum replacement budget, or price floors.
- **0 viable:** When one player dominates value (pts/price), no one else is within 10%. Increase diversity_compression or value_tolerance.

---

## Findings

1. **Episode-reactive pricing** — Prices move each episode based on performance (immunity, vote matched, etc.).
2. **Diversity compression** — Pulling toward median creates price bands so multiple players cluster at similar values.
3. **Replacement diversity** — When someone is voted off, the system aims for 3+ viable replacement options.
4. **No single obvious pick** — Value tolerance (10%) means several players can be 'best' within margin.
5. **Tuning** — Increase `diversity_compression` for more viable options; increase `price_reactivity` for faster price movement.


# Dynamic Pricing Simulation Report

## Overview

This report documents how **episode-reactive price changes** work and whether they create **multiple viable replacement options** when a player gets voted off.

**Goal:** When someone is voted off and teams need to replace them, there should NOT be one obvious player everyone wants. Multiple options at different price points should be viable.

---

## How Price Changes Work

### Update Formula

After each episode:

1. **Points earned** — Each contestant gets points for: survival, immunity, vote matched, etc.
2. **Relative performance** — Compare each player's points to the episode average.
3. **Price delta** — `(points - avg) / range × reactivity`
   - Strong performers (above avg): price increases
   - Weak performers (below avg): price decreases
4. **Dynamic diversity compression** — Compression scales up as season progresses (base→late).
   - Reduces one player becoming dominant; creates value bands; stronger in late-season.
5. **Season inflation** — Each tribal episode, prices multiply by ~1.059 so they are ~2x by merge.
6. **Dynamic bounds** — price_min/max scale with season progress (2x at merge: 160k-520k).

### Key Parameters

| Parameter | Value | Effect |
|-----------|-------|--------|
| price_reactivity | 0.15 | Max % price move per episode |
| diversity_compression (base→late) | 0.25→0.45 | Scales up as season progresses |
| value_tolerance (base→max) | 0.1→0.18 | Adaptive: widens when compressed or late-season |
| merge_price_multiplier | 2.0 | Prices ~2x by merge |
| merge_episodes | 12 | Tribal episodes before merge |
| merge_budget | $1,780,000 | Budget at merge (tuned for ~70% valid) |

---

## Replacement Diversity Results

When a player is voted off, we count how many replacement options are **viable** (within adaptive tolerance of best value, affordable with freed budget).

| Metric | Value |
|--------|-------|
| Replacement events simulated | 1,050 |
| Avg viable options per event | 2.0 |
| Min viable options (worst case) | 0 |
| % of events with 3+ viable options | 19.2% |

---

## Merge Budget Pressure (10-12 Players Left)

When the season reaches merge (10-12 players remaining), prices have inflated ~2x. We measure what % of 7-player tribe-valid combos are under budget.

| Metric | Value |
|--------|-------|
| Merge events simulated | 150 |
| Avg % valid at merge | 70.3% |
| Target % valid | 70% |

---

## Example Replacement Scenarios

### Event 1: c07 voted off (Episode 1)

- **Viable replacements:** 10
- **Affordable options:** 19
- **Viable picks:** c22, c02, c24, c19, c04, c20, c12, c14...

### Event 2: c06 voted off (Episode 2)

- **Viable replacements:** 0
- **Affordable options:** 0

### Event 3: c18 voted off (Episode 3)

- **Viable replacements:** 7
- **Affordable options:** 7
- **Viable picks:** c22, c24, c04, c20, c12, c14, c16

### Event 4: c17 voted off (Episode 4)

- **Viable replacements:** 10
- **Affordable options:** 10
- **Viable picks:** c22, c02, c24, c19, c04, c03, c20, c12...

### Event 5: c22 voted off (Episode 5)

- **Viable replacements:** 0
- **Affordable options:** 0

### Event 6: c03 voted off (Episode 6)

- **Viable replacements:** 0
- **Affordable options:** 0

### Event 7: c11 voted off (Episode 7)

- **Viable replacements:** 1
- **Affordable options:** 1
- **Viable picks:** c16

### Event 8: c01 voted off (Episode 8)

- **Viable replacements:** 0
- **Affordable options:** 0

### Event 9: c13 voted off (Episode 9)

- **Viable replacements:** 2
- **Affordable options:** 2
- **Viable picks:** c12, c16

### Event 10: c16 voted off (Episode 10)

- **Viable replacements:** 0
- **Affordable options:** 0

---

## Price Evolution (Sample Scenario)

**After Episode 0:** c01=$125,000, c02=$95,000, c03=$105,000, c04=$85,000, c05=$175,000
**After Episode 1:** c01=$132,500, c02=$110,000, c03=$117,500, c04=$100,000, c05=$172,500
**After Episode 2:** c01=$137,500, c02=$122,500, c03=$125,000, c04=$112,500, c05=$172,500
**After Episode 3:** c01=$150,000, c02=$132,500, c03=$135,000, c04=$127,500, c05=$175,000
**After Episode 4:** c01=$157,500, c02=$142,500, c03=$142,500, c04=$140,000, c05=$172,500

---

## Price Change Formula Example

When a player wins individual immunity (+5 pts) and another gets vote matched (+2 pts) while the tribe loses immunity (-1):

- **Immunity winner:** pts = 5, avg ≈ 1.5 → delta_norm positive → price goes **up**
- **Vote matched:** pts = 2, avg ≈ 1.5 → delta_norm slightly positive → price goes **up** slightly
- **Tribe lost immunity:** pts = -1, avg ≈ 1.5 → delta_norm negative → price goes **down**
- **Voted out:** pts = -8 (e.g.), avg ≈ 1.5 → delta_norm very negative → price drops (but they're removed from pool)

Dynamic diversity compression pulls prices toward the median (scaling up as the season progresses), so similar performers cluster at similar prices.

---

## Edge Cases

- **0 affordable:** When a cheap player is voted off, budget freed is low. If all remaining players cost more, no one is affordable. Consider: minimum replacement budget, or price floors.
- **0 viable:** When one player dominates value (pts/price), no one else is within tolerance. Adaptive tolerance widens when compressed or late-season.

---

## Findings

1. **Episode-reactive pricing** — Prices move each episode based on performance (immunity, vote matched, etc.).
2. **Diversity compression** — Pulling toward median creates price bands so multiple players cluster at similar values.
3. **Replacement diversity** — When someone is voted off, the system aims for 3+ viable replacement options.
4. **Adaptive value tolerance** — Widens when prices are highly compressed or in late-season states, reducing degenerate 'one obvious pick' outcomes while preserving scarcity and budget pressure.
5. **Edge cases preserved** — Impossible immediate replacement (0 affordable) remains; forces meaningful tradeoff decisions rather than auto-fills.
6. **Merge price inflation** — By merge (10-12 players), prices are ~2x start; ~70% of combos valid under budget.


# Survivor Fantasy Simulation — Compiled Report

**Generated:** February 19, 2025  
**Purpose:** Consolidated output from pricing and dynamic pricing simulations for Survivor S50 fantasy system.

---

## Executive Summary

| Metric | Value |
|--------|-------|
| **Total valid team options** | 234,727 |
| **Excluded by pricing** | 24.7% |
| **Merge valid % (target 70%)** | 70.3% |
| **Price range** | $85,000 – $217,500 |
| **Budget** | $1,000,000 |

---

## 1. Pricing System Overview

### Initial Pricing (Pre-Season)

- **Roster rules:** Exactly 7 players, minimum 1 per tribe (Tribe A, B, C)
- **Price bounds:** $80,000 – $260,000 (increments of $2,500)
- **Calibration:** Top 7 by expected points sum to $1,270,000 (exceeds budget) — cannot own all 7 most expensive
- **Target:** ~75% of tribe-valid combos under budget; ~25% excluded

### Price Distribution (Current Run)

| Metric | Value |
|--------|-------|
| Min price | $85,000 |
| Max price | $217,500 |
| Top 5 cost | $965,000 |
| Top 6 cost | $1,117,500 |
| Top 7 cost | $1,270,000 |

### Players by Price Tier

| Price | Contestants |
|-------|-------------|
| $217,500 | c21, c05 |
| $197,500 | c13 |
| $170,000 | c23 |
| $162,500 | c11 |
| $152,500 | c17, c07 |
| $150,000 | c15, c19 |
| $147,500 | c01 |
| $145,000 | c09 |
| $142,500 | c03 |
| $122,500 | c08 |
| $120,000 | c18, c10, c24 |
| $107,500 | c02, c16 |
| $87,500 | c20, c04 |
| $85,000 | c12, c06, c22, c14 |

---

## 2. Dynamic (Episode-Reactive) Pricing

### How It Works

After each tribal episode:

1. **Performance delta** — Prices move based on points earned (survival, immunity, vote matched, etc.)
2. **Diversity compression** — Pulls prices toward median (scales up as season progresses)
3. **Season inflation** — Multiply by ~1.059 per episode → ~2x by merge
4. **Dynamic bounds** — price_min/max scale with progress (2x at merge: 160k–520k)

### Key Parameters

| Parameter | Value |
|-----------|-------|
| price_reactivity | 0.15 |
| diversity_compression (base→late) | 0.25 → 0.45 |
| value_tolerance (base→max) | 0.10 → 0.18 |
| merge_price_multiplier | 2.0 |
| merge_episodes | 12 |
| merge_budget | $1,780,000 |

### Replacement Diversity Results

| Metric | Value |
|--------|-------|
| Replacement events simulated | 1,050 |
| Avg viable options per event | 2.0 |
| % of events with 3+ viable options | 19.2% |
| Merge events (10–12 players) | 150 |
| Avg % valid at merge | 70.3% |

---

## 3. Point Scoring Breakdown

### Category Totals (Aggregate)

| Category | Total Points | % of Total |
|----------|--------------|------------|
| Survival | 9,300,931 | 42.0% |
| Challenges | 8,079,576 | 36.5% |
| Tribal | 6,385,422 | 28.9% |
| Advantages | 1,422,548 | 6.4% |
| Placement | 391,979 | 1.8% |
| Penalties | -3,457,014 | -15.6% |

### Event-Level Breakdown

| Event Type | Total Points | % of Total |
|------------|--------------|------------|
| Vote matched | 6,385,422 | 28.9% |
| Survival (pre-merge) | 4,477,218 | 20.2% |
| Team immunity 1st | 4,011,582 | 18.1% |
| Survival (post-merge) | 2,749,977 | 12.4% |
| Team reward 1st | 2,663,272 | 12.0% |
| Survival (swap) | 2,073,736 | 9.4% |
| Strategic player | 1,256,368 | 5.7% |
| Individual immunity | 620,300 | 2.8% |
| Final tribal | 245,495 | 1.1% |
| Win season | 146,484 | 0.7% |
| Idol play | 80,052 | 0.4% |
| Clue read | 57,240 | 0.3% |
| Advantage play | 28,888 | 0.1% |
| Idol failure | -20,820 | -0.1% |
| Team immunity last | -782,630 | -3.5% |
| Voted out | -3,436,194 | -15.5% |

---

## 4. Roster Strategies & Performance

| Strategy | Avg Score | Avg Cost | Description |
|----------|-----------|----------|-------------|
| **random** | 411.6 | $899,500 | Random valid rosters under budget |
| **mid_tier** | 408.3 | $840,000 | Middle 50% expected points only |
| **stars_and_scrubs** | 405.2 | $862,500 | 2 stars + cheapest fill |
| **value** | 371.2 | $622,500 | Best points-per-dollar |
| **max_expected** | 338.4 | $965,000 | Highest expected points |
| **balanced** | 338.4 | $965,000 | Value with expected tiebreaker |
| **five_premium** | 338.4 | $965,000 | Exactly 5 premium players |
| **six_premium** | 338.4 | $965,000 | Exactly 6 premium players |

**Note:** Premium-heavy strategies (max_expected, five_premium, six_premium) underperform value and mid-tier strategies in this simulation, as they spend more on fewer players who may be voted out early.

---

## 5. Simulation Parameters

### Pricing Simulation

- **Price estimation runs:** 2,000 Monte Carlo scenarios
- **Scenario runs (scoring):** 300
- **Unique roster compositions tested:** 29
- **Roster strategies:** 8 (max_expected, value, balanced, mid_tier, stars_and_scrubs, five_premium, six_premium, random)

### Dynamic Pricing Simulation

- **Scenarios:** 50 full seasons
- **Merge detection:** When 10–12 players remain
- **Budget at merge:** $1,780,000 (tuned for ~70% valid)

---

## 6. Key Findings

1. **Pricing constraint achieved** — Top 7 most expensive exceed budget; ~25% of combos excluded.
2. **Merge inflation on target** — 70.3% of merge combos valid under $1.78M budget.
3. **Replacement diversity** — 19.2% of vote-off events have 3+ viable replacements; edge cases (0 affordable) preserved for tradeoff decisions.
4. **Point distribution** — Survival (42%) and challenges (36.5%) dominate; voted-out penalty (-15.5%) is significant.
5. **Strategy variance** — Value and mid-tier strategies outperform premium-heavy builds in this no-replacement model.

---

## 7. Source Files

| Report | Description |
|--------|-------------|
| `pricing_report.md` | Phase 1 pricing simulation output |
| `DYNAMIC_PRICING_REPORT.md` | Episode-reactive pricing and replacement diversity |
| `pricing_analysis.json` | Raw pricing and strategy stats |
| `dynamic_pricing_analysis.json` | Raw dynamic pricing stats |
| `report.md` | Point simulation methodology |


# Full-Stack Simulation Report

## Overview

**Total runs:** 4,000 (scenarios × rosters × play styles)
**Scenarios:** 50
**Replacement penalty:** -10 points per add
**Captain multiplier:** 2.0x (required every episode)

---

## Executive Summary for Tuning

| Metric | Value | Tuning implication |
|--------|-------|-------------------|
| Survival % of total | 39.3% | Core floor; adjust pre/post/swap if too flat |
| Challenges % of total | 32.2% | Immunity/reward weights; individual immunity rare |
| Vote matched % | 26.3% | High impact; consider if over-rewarding |
| Voted out penalty % | -17.7% | Major negative; pocket multiplier matters |
| Captain bonus % | 14.3% | 2x adds ~64 pts/run avg |
| Replacement lift | +59.0 pts vs fixed | Replace worth it despite -10 penalty |
| Best combo | random_replace | Target for balance; others should close gap |
| Worst combo | max_expected_fixed | May need pricing/strategy tweaks |

---

## How the Simulation Works

### Rules of Each Simulated Game

1. **Initial roster:** 7 players, min 1 per tribe, under $1M budget. Built using strategy (value, max_expected, mid_tier, etc.).
2. **No replacement (fixed):** When a roster member is voted off, they are not replaced. The roster shrinks; eliminated players stop earning points but can still incur voted-out penalty.
3. **Replacement (replace):** When a roster member is voted off, if viable replacements exist (affordable with freed budget, within value tolerance), the best-value option is added. A **-10 point penalty** is applied per replacement.
4. **Captain (required):** Every episode, captain = highest expected pts among active roster. Captain earns **2x points** for that episode only.
5. **Price evolution:** Prices update after each tribal (performance delta, diversity compression, season inflation). All rosters in a scenario see the same price evolution.
6. **Scoring:** Survival, challenges, tribal, advantages, placement. Same point values as production config.

### Play Styles Tested

| Style | Replacement | Captain |
|-------|-------------|---------|
| fixed | No | Yes (required) |
| replace | Yes (when viable) | Yes (required) |

---

## Point Breakdown (Aggregate)

### Survival: 709,378 points (39.3%)

- **survival_pre_merge:** 304,704 pts (152,352 events, 16.9%)
- **survival_swap:** 162,934 pts (81,467 events, 9.0%)
- **survival_post_merge:** 241,740 pts (80,580 events, 13.4%)

### Challenges: 581,837 points (32.2%)

- **team_immunity_first:** 277,980 pts (92,660 events, 15.4%)
- **team_immunity_second_three:** 52,978 pts (52,978 events, 2.9%)
- **team_immunity_second_two:** 0 pts (42,513 events, 0.0%)
- **team_immunity_last:** -55,336 pts (55,336 events, -3.1%)
- **team_reward_first:** 194,486 pts (97,243 events, 10.8%)
- **team_reward_second_three:** 51,574 pts (51,574 events, 2.9%)
- **team_reward_second_two:** 0 pts (43,112 events, 0.0%)
- **individual_immunity:** 60,155 pts (12,031 events, 3.3%)

### Tribal: 154,176 points (8.5%)

- **vote_matched:** 474,102 pts (237,051 events, 26.3%)
- **voted_out:** -319,926 pts (18,309 events, -17.7%)

### Advantages: 124,892 points (6.9%)

- **clue_read:** 6,340 pts (6,340 events, 0.4%)
- **advantage_play:** 2,540 pts (635 events, 0.1%)
- **idol_play:** 8,806 pts (1,258 events, 0.5%)
- **idol_failure:** -1,106 pts (553 events, -0.1%)
- **strategic_player:** 108,312 pts (27,078 events, 6.0%)

### Placement: 41,324 points (2.3%)

- **final_tribal:** 26,015 pts (5,203 events, 1.4%)
- **win_season:** 15,309 pts (1,701 events, 0.8%)

### Other: 0 points (0.0%)


### Captain Bonus: 257,983 points
### Replacement Penalty: -63,650 points

---

## Play Style Comparison

| Play Style | Avg Score | Runs |
|------------|-----------|------|
| fixed | 422.0 | 2,000 |
| replace | 481.0 | 2,000 |

---

## Strategy × Play Style (Emerging Strategies)

| Strategy | Play Style | Avg | Min | Max |
|----------|------------|-----|-----|-----|
| balanced | fixed | 403.3 | 206.0 | 620.0 |
| balanced | replace | 485.7 | 163.0 | 753.0 |
| five_premium | fixed | 403.3 | 206.0 | 620.0 |
| five_premium | replace | 485.7 | 163.0 | 753.0 |
| max_expected | fixed | 403.3 | 206.0 | 620.0 |
| max_expected | replace | 485.7 | 163.0 | 753.0 |
| mid_tier | fixed | 427.4 | 200.0 | 711.0 |
| mid_tier | replace | 474.0 | 200.0 | 808.0 |
| random | fixed | 446.2 | 174.0 | 812.0 |
| random | replace | 501.9 | 157.0 | 835.0 |
| six_premium | fixed | 403.3 | 206.0 | 620.0 |
| six_premium | replace | 485.7 | 163.0 | 753.0 |
| stars_and_scrubs | fixed | 446.5 | 183.0 | 691.0 |
| stars_and_scrubs | replace | 462.1 | 241.0 | 698.0 |
| value | fixed | 442.4 | 93.0 | 682.0 |
| value | replace | 467.2 | 93.0 | 704.0 |

---

## Captaincy Impact

Total captain bonus across all runs: **257,983** points (~64 per run).
Captain is required every episode; chosen as highest expected pts among active roster.
**Tuning:** If captain bonus dominates, lower multiplier (e.g. 1.5x). If too weak, raise it.

---

## Tuning Recommendations

### Point System
- **Survival vs challenges:** If survival dominates (>45%), consider raising challenge pts (immunity, reward).
- **Vote matched:** Often highest positive event. If you want less predictability, lower it.
- **Voted out:** Pocket multiplier (2^items) can create huge swings. Consider capping or flattening.
- **Strategic player:** High variance event. Adjust if too swingy.

### Pricing
- **price_min/max:** Wider range = more roster diversity but harder to balance.
- **target_valid_pct:** Lower = fewer viable combos = more budget pressure.
- **Replacement penalty:** -10 tested here. If replace always dominates, raise penalty.

### Strategy Balance
- Best: **random_replace** (avg 501.9). Worst: **max_expected_fixed** (avg 403.3).
- Gap: 98.6 pts. Narrow by: flattening prices, adjusting point weights, or roster constraints.

---

## Roster Change Viability

**Replacements made:** 6,365
**Total replacement penalty:** -63,650 points
**Penalty per replacement:** -10

Replacements occur when a roster member is voted off and viable options exist
(affordable with freed budget, within value tolerance). Constant roster changes
are viable when replacements are affordable; the -10 point penalty creates
tradeoff between keeping a weaker roster vs. paying to upgrade.

---

## Price Changes: How Big of a Deal?

Prices update after each tribal via:
- **Performance delta:** Strong performers (immunity, vote matched) rise; weak performers fall
- **Diversity compression:** Pulls prices toward median to create value bands
- **Season inflation:** ~1.059x per episode → ~2x by merge

**Impact on replacements:** When a roster member is voted off, budget freed = their price.
Higher-priced vote-offs (e.g. $200k) free more budget for replacements than cheap vote-offs ($100k).
Early cheap vote-offs often yield 0 affordable replacements; late expensive vote-offs
free budget but fewer players remain. Price dynamics create strategic tension:
owning expensive players risks early vote-off with limited replacement options.

**Impact on strategy:** Replace outperforms fixed by **+59.0 pts** on average,
despite the -10 penalty per replacement. If this gap is too large, consider:
raising replacement penalty, tightening value tolerance, or reducing merge budget.

---

## Highest Potential Captaincies

Contestants with highest expected points offer the most captain upside:

- **c21:** 66.3 expected pts (2x when captained)
- **c23:** 63.1 expected pts (2x when captained)
- **c05:** 62.4 expected pts (2x when captained)
- **c13:** 60.8 expected pts (2x when captained)
- **c07:** 60.2 expected pts (2x when captained)
- **c15:** 58.4 expected pts (2x when captained)
- **c11:** 58.4 expected pts (2x when captained)
- **c09:** 57.9 expected pts (2x when captained)
- **c17:** 57.4 expected pts (2x when captained)
- **c01:** 56.7 expected pts (2x when captained)


# Episode Trace: Week-by-Week Simulation

**Scenario seed:** 99 — run `python run_episode_trace_simulation.py --seed 123` for different outcomes.

Shows how prices change, who gets voted out, and how fantasy teams are affected.

---

## Sample Teams (Final Results)

### value

- **Fixed (no replace):** 422 pts — roster: c04, c12, c22, c20, c16, c18, c10
- **Replace (when viable):** 481 pts — roster: c04, c12, c22, c20, c16, c18, c20
- Initial cost: $787,500 / $1,000,000 budget

### mid_tier

- **Fixed (no replace):** 612 pts — roster: c02, c16, c18, c10, c24, c15, c09
- **Replace (when viable):** 674 pts — roster: c02, c16, c18, c24, c15, c09, c20
- Initial cost: $907,500 / $1,000,000 budget

### random

- **Fixed (no replace):** 424 pts — roster: c02, c12, c23, c16, c04, c18, c05
- **Replace (when viable):** 423 pts — roster: c02, c12, c23, c16, c04, c18, c23
- Initial cost: $917,500 / $1,000,000 budget

---

## Week-by-Week Breakdown

### Episode 1 (pre_merge)

**Voted out:** c05 (Contestant 05)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c05 (Contestant 05) | $182,500 | $165,000 | -17,500 (-9.6%) |
| c14 (Contestant 14) | $102,500 | $120,000 | +17,500 (+17.1%) |
| c20 (Contestant 20) | $102,500 | $120,000 | +17,500 (+17.1%) |
| c22 (Contestant 22) | $102,500 | $120,000 | +17,500 (+17.1%) |
| c06 (Contestant 06) | $102,500 | $117,500 | +15,000 (+14.6%) |
| c12 (Contestant 12) | $110,000 | $125,000 | +15,000 (+13.6%) |
| c04 (Contestant 04) | $102,500 | $115,000 | +12,500 (+12.2%) |
| c10 (Contestant 10) | $125,000 | $137,500 | +12,500 (+10.0%) |
| c16 (Contestant 16) | $120,000 | $132,500 | +12,500 (+10.4%) |
| c18 (Contestant 18) | $125,000 | $137,500 | +12,500 (+10.0%) |
| c09 (Contestant 09) | $135,000 | $145,000 | +10,000 (+7.4%) |
| c15 (Contestant 15) | $135,000 | $145,000 | +10,000 (+7.4%) |
| ... | (11 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 56.0 | 56.0 | — | c18 |
| value | replace | 56.0 | 56.0 | — | c18 |
| mid_tier | fixed | 48.0 | 48.0 | — | c02 |
| mid_tier | replace | 48.0 | 48.0 | — | c02 |
| random | fixed | 2.0 | 2.0 | — | c05 |
| random | replace | 2.0 | -8.0 | c05 → c23 ($182,500 freed, 17 viable) | c05 |

---

### Episode 2 (pre_merge)

**Voted out:** c06 (Contestant 06)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c04 (Contestant 04) | $115,000 | $132,500 | +17,500 (+15.2%) |
| c14 (Contestant 14) | $120,000 | $135,000 | +15,000 (+12.5%) |
| c20 (Contestant 20) | $120,000 | $135,000 | +15,000 (+12.5%) |
| c22 (Contestant 22) | $120,000 | $135,000 | +15,000 (+12.5%) |
| c02 (Contestant 02) | $147,500 | $160,000 | +12,500 (+8.5%) |
| c12 (Contestant 12) | $125,000 | $137,500 | +12,500 (+10.0%) |
| c16 (Contestant 16) | $132,500 | $145,000 | +12,500 (+9.4%) |
| c18 (Contestant 18) | $137,500 | $150,000 | +12,500 (+9.1%) |
| c03 (Contestant 03) | $147,500 | $157,500 | +10,000 (+6.8%) |
| c09 (Contestant 09) | $145,000 | $155,000 | +10,000 (+6.9%) |
| c10 (Contestant 10) | $137,500 | $147,500 | +10,000 (+7.3%) |
| c15 (Contestant 15) | $145,000 | $155,000 | +10,000 (+6.9%) |
| ... | (11 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 46.0 | 102.0 | — | c18 |
| value | replace | 46.0 | 102.0 | — | c18 |
| mid_tier | fixed | 52.0 | 100.0 | — | c02 |
| mid_tier | replace | 52.0 | 100.0 | — | c02 |
| random | fixed | 51.0 | 53.0 | — | c02 |
| random | replace | 57.0 | 49.0 | — | c02 |

---

### Episode 3 (pre_merge)

**Voted out:** c19 (Contestant 19)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c06 (Contestant 06) | $122,500 | $140,000 | +17,500 (+14.3%) |
| c14 (Contestant 14) | $135,000 | $152,500 | +17,500 (+13.0%) |
| c04 (Contestant 04) | $132,500 | $147,500 | +15,000 (+11.3%) |
| c12 (Contestant 12) | $137,500 | $152,500 | +15,000 (+10.9%) |
| c15 (Contestant 15) | $155,000 | $170,000 | +15,000 (+9.7%) |
| c16 (Contestant 16) | $145,000 | $160,000 | +15,000 (+10.3%) |
| c09 (Contestant 09) | $155,000 | $167,500 | +12,500 (+8.1%) |
| c10 (Contestant 10) | $147,500 | $160,000 | +12,500 (+8.5%) |
| c18 (Contestant 18) | $150,000 | $162,500 | +12,500 (+8.3%) |
| c20 (Contestant 20) | $135,000 | $147,500 | +12,500 (+9.3%) |
| c22 (Contestant 22) | $135,000 | $147,500 | +12,500 (+9.3%) |
| c24 (Contestant 24) | $147,500 | $160,000 | +12,500 (+8.5%) |
| ... | (11 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 45.0 | 147.0 | — | c18 |
| value | replace | 45.0 | 147.0 | — | c18 |
| mid_tier | fixed | 52.0 | 152.0 | — | c02 |
| mid_tier | replace | 52.0 | 152.0 | — | c02 |
| random | fixed | 35.0 | 88.0 | — | c02 |
| random | replace | 38.0 | 87.0 | — | c02 |

---

### Episode 4 (pre_merge)

**Voted out:** c10 (Contestant 10)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c04 (Contestant 04) | $147,500 | $165,000 | +17,500 (+11.9%) |
| c06 (Contestant 06) | $140,000 | $155,000 | +15,000 (+10.7%) |
| c12 (Contestant 12) | $152,500 | $167,500 | +15,000 (+9.8%) |
| c14 (Contestant 14) | $152,500 | $167,500 | +15,000 (+9.8%) |
| c16 (Contestant 16) | $160,000 | $175,000 | +15,000 (+9.4%) |
| c01 (Contestant 01) | $172,500 | $185,000 | +12,500 (+7.2%) |
| c02 (Contestant 02) | $167,500 | $180,000 | +12,500 (+7.5%) |
| c03 (Contestant 03) | $167,500 | $180,000 | +12,500 (+7.5%) |
| c08 (Contestant 08) | $172,500 | $185,000 | +12,500 (+7.2%) |
| c20 (Contestant 20) | $147,500 | $160,000 | +12,500 (+8.5%) |
| c22 (Contestant 22) | $147,500 | $160,000 | +12,500 (+8.5%) |
| c07 (Contestant 07) | $177,500 | $187,500 | +10,000 (+5.6%) |
| ... | (11 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 25.0 | 172.0 | — | c18 |
| value | replace | 25.0 | 162.0 | c10 → c20 ($160,000 freed, 2 viable) | c18 |
| mid_tier | fixed | 32.0 | 184.0 | — | c02 |
| mid_tier | replace | 32.0 | 174.0 | c10 → c20 ($160,000 freed, 2 viable) | c02 |
| random | fixed | 49.0 | 137.0 | — | c02 |
| random | replace | 52.0 | 139.0 | — | c02 |

---

### Episode 5 (pre_merge)

**Voted out:** c17 (Contestant 17)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c24 (Contestant 24) | $170,000 | $187,500 | +17,500 (+10.3%) |
| c06 (Contestant 06) | $155,000 | $170,000 | +15,000 (+9.7%) |
| c10 (Contestant 10) | $157,500 | $172,500 | +15,000 (+9.5%) |
| c12 (Contestant 12) | $167,500 | $182,500 | +15,000 (+9.0%) |
| c14 (Contestant 14) | $167,500 | $182,500 | +15,000 (+9.0%) |
| c22 (Contestant 22) | $160,000 | $175,000 | +15,000 (+9.4%) |
| c04 (Contestant 04) | $165,000 | $177,500 | +12,500 (+7.6%) |
| c09 (Contestant 09) | $177,500 | $190,000 | +12,500 (+7.0%) |
| c15 (Contestant 15) | $180,000 | $192,500 | +12,500 (+6.9%) |
| c16 (Contestant 16) | $175,000 | $187,500 | +12,500 (+7.1%) |
| c18 (Contestant 18) | $172,500 | $185,000 | +12,500 (+7.2%) |
| c19 (Contestant 19) | $172,500 | $185,000 | +12,500 (+7.2%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 41.0 | 213.0 | — | c18 |
| value | replace | 45.0 | 207.0 | — | c18 |
| mid_tier | fixed | 47.0 | 231.0 | — | c02 |
| mid_tier | replace | 51.0 | 225.0 | — | c02 |
| random | fixed | 41.0 | 178.0 | — | c02 |
| random | replace | 47.0 | 186.0 | — | c02 |

---

### Episode 6 (pre_merge)

**Voted out:** c14 (Contestant 14)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c20 (Contestant 20) | $172,500 | $190,000 | +17,500 (+10.1%) |
| c22 (Contestant 22) | $175,000 | $192,500 | +17,500 (+10.0%) |
| c06 (Contestant 06) | $170,000 | $185,000 | +15,000 (+8.8%) |
| c10 (Contestant 10) | $172,500 | $187,500 | +15,000 (+8.7%) |
| c12 (Contestant 12) | $182,500 | $197,500 | +15,000 (+8.2%) |
| c16 (Contestant 16) | $187,500 | $202,500 | +15,000 (+8.0%) |
| c17 (Contestant 17) | $172,500 | $187,500 | +15,000 (+8.7%) |
| c18 (Contestant 18) | $185,000 | $200,000 | +15,000 (+8.1%) |
| c23 (Contestant 23) | $185,000 | $200,000 | +15,000 (+8.1%) |
| c04 (Contestant 04) | $177,500 | $190,000 | +12,500 (+7.0%) |
| c08 (Contestant 08) | $192,500 | $205,000 | +12,500 (+6.5%) |
| c09 (Contestant 09) | $190,000 | $202,500 | +12,500 (+6.6%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 49.0 | 262.0 | — | c18 |
| value | replace | 57.0 | 264.0 | — | c18 |
| mid_tier | fixed | 41.0 | 272.0 | — | c02 |
| mid_tier | replace | 49.0 | 274.0 | — | c02 |
| random | fixed | 39.0 | 217.0 | — | c02 |
| random | replace | 47.0 | 233.0 | — | c02 |

---

### Episode 7 (pre_merge)

**Voted out:** c23 (Contestant 23)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c04 (Contestant 04) | $190,000 | $207,500 | +17,500 (+9.2%) |
| c06 (Contestant 06) | $185,000 | $202,500 | +17,500 (+9.5%) |
| c14 (Contestant 14) | $177,500 | $195,000 | +17,500 (+9.9%) |
| c02 (Contestant 02) | $200,000 | $215,000 | +15,000 (+7.5%) |
| c03 (Contestant 03) | $197,500 | $212,500 | +15,000 (+7.6%) |
| c07 (Contestant 07) | $202,500 | $217,500 | +15,000 (+7.4%) |
| c10 (Contestant 10) | $187,500 | $202,500 | +15,000 (+8.0%) |
| c17 (Contestant 17) | $187,500 | $202,500 | +15,000 (+8.0%) |
| c18 (Contestant 18) | $200,000 | $215,000 | +15,000 (+7.5%) |
| c20 (Contestant 20) | $190,000 | $205,000 | +15,000 (+7.9%) |
| c22 (Contestant 22) | $192,500 | $207,500 | +15,000 (+7.8%) |
| c01 (Contestant 01) | $200,000 | $212,500 | +12,500 (+6.2%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 46.0 | 308.0 | — | c18 |
| value | replace | 51.0 | 315.0 | — | c18 |
| mid_tier | fixed | 45.0 | 317.0 | — | c02 |
| mid_tier | replace | 50.0 | 324.0 | — | c02 |
| random | fixed | 26.0 | 243.0 | — | c02 |
| random | replace | 9.0 | 242.0 | — | c02 |

---

### Episode 8 (pre_merge)

**Voted out:** c12 (Contestant 12)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c04 (Contestant 04) | $207,500 | $230,000 | +22,500 (+10.8%) |
| c02 (Contestant 02) | $215,000 | $232,500 | +17,500 (+8.1%) |
| c03 (Contestant 03) | $212,500 | $230,000 | +17,500 (+8.2%) |
| c06 (Contestant 06) | $202,500 | $220,000 | +17,500 (+8.6%) |
| c10 (Contestant 10) | $202,500 | $220,000 | +17,500 (+8.6%) |
| c14 (Contestant 14) | $195,000 | $212,500 | +17,500 (+9.0%) |
| c17 (Contestant 17) | $202,500 | $220,000 | +17,500 (+8.6%) |
| c23 (Contestant 23) | $195,000 | $212,500 | +17,500 (+9.0%) |
| c01 (Contestant 01) | $212,500 | $227,500 | +15,000 (+7.1%) |
| c07 (Contestant 07) | $217,500 | $232,500 | +15,000 (+6.9%) |
| c08 (Contestant 08) | $217,500 | $232,500 | +15,000 (+6.9%) |
| c19 (Contestant 19) | $210,000 | $225,000 | +15,000 (+7.1%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 15.0 | 323.0 | — | c18 |
| value | replace | 19.0 | 334.0 | — | c18 |
| mid_tier | fixed | 37.0 | 354.0 | — | c02 |
| mid_tier | replace | 41.0 | 365.0 | — | c02 |
| random | fixed | 23.0 | 266.0 | — | c02 |
| random | replace | 23.0 | 265.0 | — | c02 |

---

### Episode 9 (swap)

**Voted out:** c11 (Contestant 11)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c09 (Contestant 09) | $227,500 | $250,000 | +22,500 (+9.9%) |
| c12 (Contestant 12) | $207,500 | $227,500 | +20,000 (+9.6%) |
| c22 (Contestant 22) | $220,000 | $240,000 | +20,000 (+9.1%) |
| c03 (Contestant 03) | $230,000 | $247,500 | +17,500 (+7.6%) |
| c14 (Contestant 14) | $212,500 | $230,000 | +17,500 (+8.2%) |
| c21 (Contestant 21) | $230,000 | $247,500 | +17,500 (+7.6%) |
| c23 (Contestant 23) | $212,500 | $230,000 | +17,500 (+8.2%) |
| c02 (Contestant 02) | $232,500 | $247,500 | +15,000 (+6.5%) |
| c06 (Contestant 06) | $220,000 | $235,000 | +15,000 (+6.8%) |
| c10 (Contestant 10) | $220,000 | $235,000 | +15,000 (+6.8%) |
| c16 (Contestant 16) | $227,500 | $242,500 | +15,000 (+6.6%) |
| c17 (Contestant 17) | $220,000 | $235,000 | +15,000 (+6.8%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 36.0 | 359.0 | — | c18 |
| value | replace | 40.0 | 374.0 | — | c18 |
| mid_tier | fixed | 51.0 | 405.0 | — | c02 |
| mid_tier | replace | 55.0 | 420.0 | — | c02 |
| random | fixed | 34.0 | 300.0 | — | c02 |
| random | replace | 34.0 | 299.0 | — | c02 |

---

### Episode 10 (swap)

**Voted out:** c15 (Contestant 15)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c11 (Contestant 11) | $225,000 | $242,500 | +17,500 (+7.8%) |
| c12 (Contestant 12) | $227,500 | $245,000 | +17,500 (+7.7%) |
| c14 (Contestant 14) | $230,000 | $247,500 | +17,500 (+7.6%) |
| c20 (Contestant 20) | $235,000 | $252,500 | +17,500 (+7.4%) |
| c21 (Contestant 21) | $247,500 | $265,000 | +17,500 (+7.1%) |
| c23 (Contestant 23) | $230,000 | $247,500 | +17,500 (+7.6%) |
| c02 (Contestant 02) | $247,500 | $262,500 | +15,000 (+6.1%) |
| c05 (Contestant 05) | $240,000 | $255,000 | +15,000 (+6.2%) |
| c06 (Contestant 06) | $235,000 | $250,000 | +15,000 (+6.4%) |
| c07 (Contestant 07) | $240,000 | $255,000 | +15,000 (+6.2%) |
| c08 (Contestant 08) | $242,500 | $257,500 | +15,000 (+6.2%) |
| c10 (Contestant 10) | $235,000 | $250,000 | +15,000 (+6.4%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 36.0 | 395.0 | — | c18 |
| value | replace | 42.0 | 416.0 | — | c18 |
| mid_tier | fixed | 29.0 | 434.0 | — | c02 |
| mid_tier | replace | 35.0 | 455.0 | — | c02 |
| random | fixed | 32.0 | 332.0 | — | c02 |
| random | replace | 32.0 | 331.0 | — | c02 |

---

### Episode 11 (swap)

**Voted out:** c22 (Contestant 22)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c18 (Contestant 18) | $257,500 | $280,000 | +22,500 (+8.7%) |
| c15 (Contestant 15) | $235,000 | $255,000 | +20,000 (+8.5%) |
| c01 (Contestant 01) | $252,500 | $270,000 | +17,500 (+6.9%) |
| c04 (Contestant 04) | $252,500 | $270,000 | +17,500 (+6.9%) |
| c11 (Contestant 11) | $242,500 | $260,000 | +17,500 (+7.2%) |
| c12 (Contestant 12) | $245,000 | $262,500 | +17,500 (+7.1%) |
| c14 (Contestant 14) | $247,500 | $265,000 | +17,500 (+7.1%) |
| c20 (Contestant 20) | $252,500 | $270,000 | +17,500 (+6.9%) |
| c23 (Contestant 23) | $247,500 | $265,000 | +17,500 (+7.1%) |
| c24 (Contestant 24) | $252,500 | $270,000 | +17,500 (+6.9%) |
| c03 (Contestant 03) | $260,000 | $275,000 | +15,000 (+5.8%) |
| c05 (Contestant 05) | $255,000 | $270,000 | +15,000 (+5.9%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 32.0 | 427.0 | — | c18 |
| value | replace | 38.0 | 454.0 | — | c18 |
| mid_tier | fixed | 41.0 | 475.0 | — | c02 |
| mid_tier | replace | 47.0 | 502.0 | — | c02 |
| random | fixed | 34.0 | 366.0 | — | c02 |
| random | replace | 34.0 | 365.0 | — | c02 |

---

### Episode 12 (swap)

**Voted out:** c04 (Contestant 04)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c15 (Contestant 15) | $255,000 | $277,500 | +22,500 (+8.8%) |
| c20 (Contestant 20) | $270,000 | $292,500 | +22,500 (+8.3%) |
| c22 (Contestant 22) | $250,000 | $272,500 | +22,500 (+9.0%) |
| c07 (Contestant 07) | $270,000 | $290,000 | +20,000 (+7.4%) |
| c08 (Contestant 08) | $272,500 | $292,500 | +20,000 (+7.3%) |
| c11 (Contestant 11) | $260,000 | $280,000 | +20,000 (+7.7%) |
| c12 (Contestant 12) | $262,500 | $282,500 | +20,000 (+7.6%) |
| c13 (Contestant 13) | $270,000 | $290,000 | +20,000 (+7.4%) |
| c24 (Contestant 24) | $270,000 | $290,000 | +20,000 (+7.4%) |
| c01 (Contestant 01) | $270,000 | $287,500 | +17,500 (+6.5%) |
| c05 (Contestant 05) | $270,000 | $287,500 | +17,500 (+6.5%) |
| c06 (Contestant 06) | $265,000 | $282,500 | +17,500 (+6.6%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 4.0 | 431.0 | — | c18 |
| value | replace | 17.0 | 471.0 | — | c18 |
| mid_tier | fixed | 29.0 | 504.0 | — | c02 |
| mid_tier | replace | 42.0 | 544.0 | — | c02 |
| random | fixed | -5.0 | 361.0 | — | c02 |
| random | replace | -5.0 | 360.0 | — | c02 |

---

### Episode 13 (post_merge)

**Voted out:** c16 (Contestant 16)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c04 (Contestant 04) | $265,000 | $290,000 | +25,000 (+9.4%) |
| c08 (Contestant 08) | $292,500 | $317,500 | +25,000 (+8.5%) |
| c22 (Contestant 22) | $272,500 | $295,000 | +22,500 (+8.3%) |
| c11 (Contestant 11) | $280,000 | $300,000 | +20,000 (+7.1%) |
| c15 (Contestant 15) | $277,500 | $297,500 | +20,000 (+7.2%) |
| c03 (Contestant 03) | $290,000 | $307,500 | +17,500 (+6.0%) |
| c05 (Contestant 05) | $287,500 | $305,000 | +17,500 (+6.1%) |
| c06 (Contestant 06) | $282,500 | $300,000 | +17,500 (+6.2%) |
| c07 (Contestant 07) | $290,000 | $307,500 | +17,500 (+6.0%) |
| c10 (Contestant 10) | $282,500 | $300,000 | +17,500 (+6.2%) |
| c12 (Contestant 12) | $282,500 | $300,000 | +17,500 (+6.2%) |
| c13 (Contestant 13) | $290,000 | $307,500 | +17,500 (+6.0%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 1.0 | 432.0 | — | c18 |
| value | replace | 6.0 | 477.0 | — | c18 |
| mid_tier | fixed | 9.0 | 513.0 | — | c02 |
| mid_tier | replace | 14.0 | 558.0 | — | c02 |
| random | fixed | -1.0 | 360.0 | — | c02 |
| random | replace | -1.0 | 359.0 | — | c02 |

---

### Episode 14 (post_merge)

**Voted out:** c21 (Contestant 21)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c09 (Contestant 09) | $307,500 | $332,500 | +25,000 (+8.1%) |
| c03 (Contestant 03) | $307,500 | $330,000 | +22,500 (+7.3%) |
| c04 (Contestant 04) | $290,000 | $312,500 | +22,500 (+7.8%) |
| c16 (Contestant 16) | $285,000 | $307,500 | +22,500 (+7.9%) |
| c15 (Contestant 15) | $297,500 | $317,500 | +20,000 (+6.7%) |
| c22 (Contestant 22) | $295,000 | $315,000 | +20,000 (+6.8%) |
| c01 (Contestant 01) | $302,500 | $320,000 | +17,500 (+5.8%) |
| c05 (Contestant 05) | $305,000 | $322,500 | +17,500 (+5.7%) |
| c06 (Contestant 06) | $300,000 | $317,500 | +17,500 (+5.8%) |
| c07 (Contestant 07) | $307,500 | $325,000 | +17,500 (+5.7%) |
| c10 (Contestant 10) | $300,000 | $317,500 | +17,500 (+5.8%) |
| c11 (Contestant 11) | $300,000 | $317,500 | +17,500 (+5.8%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 11.0 | 443.0 | — | c18 |
| value | replace | 16.0 | 493.0 | — | c18 |
| mid_tier | fixed | 24.0 | 537.0 | — | c02 |
| mid_tier | replace | 29.0 | 587.0 | — | c02 |
| random | fixed | 9.0 | 369.0 | — | c02 |
| random | replace | 9.0 | 368.0 | — | c02 |

---

### Episode 15 (post_merge)

**Voted out:** c09 (Contestant 09)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c20 (Contestant 20) | $325,000 | $350,000 | +25,000 (+7.7%) |
| c21 (Contestant 21) | $300,000 | $325,000 | +25,000 (+8.3%) |
| c16 (Contestant 16) | $307,500 | $330,000 | +22,500 (+7.3%) |
| c02 (Contestant 02) | $320,000 | $340,000 | +20,000 (+6.2%) |
| c04 (Contestant 04) | $312,500 | $332,500 | +20,000 (+6.4%) |
| c06 (Contestant 06) | $317,500 | $337,500 | +20,000 (+6.3%) |
| c08 (Contestant 08) | $332,500 | $352,500 | +20,000 (+6.0%) |
| c10 (Contestant 10) | $317,500 | $337,500 | +20,000 (+6.3%) |
| c11 (Contestant 11) | $317,500 | $337,500 | +20,000 (+6.3%) |
| c12 (Contestant 12) | $317,500 | $337,500 | +20,000 (+6.3%) |
| c14 (Contestant 14) | $317,500 | $337,500 | +20,000 (+6.3%) |
| c15 (Contestant 15) | $317,500 | $337,500 | +20,000 (+6.3%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 20.0 | 463.0 | — | c18 |
| value | replace | 30.0 | 523.0 | — | c18 |
| mid_tier | fixed | 4.0 | 541.0 | — | c02 |
| mid_tier | replace | 14.0 | 601.0 | — | c02 |
| random | fixed | 15.0 | 384.0 | — | c02 |
| random | replace | 15.0 | 383.0 | — | c02 |

---

### Episode 16 (post_merge)

**Voted out:** c18 (Contestant 18)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c01 (Contestant 01) | $337,500 | $365,000 | +27,500 (+8.1%) |
| c09 (Contestant 09) | $322,500 | $347,500 | +25,000 (+7.8%) |
| c21 (Contestant 21) | $325,000 | $350,000 | +25,000 (+7.7%) |
| c24 (Contestant 24) | $342,500 | $367,500 | +25,000 (+7.3%) |
| c04 (Contestant 04) | $332,500 | $355,000 | +22,500 (+6.8%) |
| c16 (Contestant 16) | $330,000 | $352,500 | +22,500 (+6.8%) |
| c02 (Contestant 02) | $340,000 | $360,000 | +20,000 (+5.9%) |
| c03 (Contestant 03) | $345,000 | $365,000 | +20,000 (+5.8%) |
| c05 (Contestant 05) | $340,000 | $360,000 | +20,000 (+5.9%) |
| c06 (Contestant 06) | $337,500 | $357,500 | +20,000 (+5.9%) |
| c07 (Contestant 07) | $342,500 | $362,500 | +20,000 (+5.8%) |
| c10 (Contestant 10) | $337,500 | $357,500 | +20,000 (+5.9%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | -35.0 | 428.0 | — | c18 |
| value | replace | -30.0 | 493.0 | — | c18 |
| mid_tier | fixed | -4.0 | 537.0 | — | c02 |
| mid_tier | replace | 1.0 | 602.0 | — | c02 |
| random | fixed | -14.0 | 370.0 | — | c02 |
| random | replace | -14.0 | 369.0 | — | c02 |

---

### Episode 17 (post_merge)

**Voted out:** c01 (Contestant 01)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c18 (Contestant 18) | $335,000 | $365,000 | +30,000 (+9.0%) |
| c13 (Contestant 13) | $360,000 | $387,500 | +27,500 (+7.6%) |
| c09 (Contestant 09) | $347,500 | $372,500 | +25,000 (+7.2%) |
| c20 (Contestant 20) | $367,500 | $392,500 | +25,000 (+6.8%) |
| c21 (Contestant 21) | $350,000 | $375,000 | +25,000 (+7.1%) |
| c03 (Contestant 03) | $365,000 | $387,500 | +22,500 (+6.2%) |
| c04 (Contestant 04) | $355,000 | $377,500 | +22,500 (+6.3%) |
| c06 (Contestant 06) | $357,500 | $380,000 | +22,500 (+6.3%) |
| c10 (Contestant 10) | $357,500 | $380,000 | +22,500 (+6.3%) |
| c11 (Contestant 11) | $357,500 | $380,000 | +22,500 (+6.3%) |
| c12 (Contestant 12) | $357,500 | $380,000 | +22,500 (+6.3%) |
| c14 (Contestant 14) | $357,500 | $380,000 | +22,500 (+6.3%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 16.0 | 444.0 | — | c20 |
| value | replace | 32.0 | 525.0 | — | c20 |
| mid_tier | fixed | 11.0 | 548.0 | — | c02 |
| mid_tier | replace | 19.0 | 621.0 | — | c02 |
| random | fixed | 6.0 | 376.0 | — | c02 |
| random | replace | 6.0 | 375.0 | — | c02 |

---

### Episode 18 (post_merge)

**Voted out:** c03 (Contestant 03)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c01 (Contestant 01) | $357,500 | $390,000 | +32,500 (+9.1%) |
| c13 (Contestant 13) | $387,500 | $415,000 | +27,500 (+7.1%) |
| c18 (Contestant 18) | $365,000 | $392,500 | +27,500 (+7.5%) |
| c20 (Contestant 20) | $392,500 | $420,000 | +27,500 (+7.0%) |
| c07 (Contestant 07) | $382,500 | $407,500 | +25,000 (+6.5%) |
| c09 (Contestant 09) | $372,500 | $397,500 | +25,000 (+6.7%) |
| c16 (Contestant 16) | $375,000 | $400,000 | +25,000 (+6.7%) |
| c21 (Contestant 21) | $375,000 | $400,000 | +25,000 (+6.7%) |
| c02 (Contestant 02) | $380,000 | $402,500 | +22,500 (+5.9%) |
| c04 (Contestant 04) | $377,500 | $400,000 | +22,500 (+6.0%) |
| c05 (Contestant 05) | $380,000 | $402,500 | +22,500 (+5.9%) |
| c06 (Contestant 06) | $380,000 | $402,500 | +22,500 (+5.9%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 20.0 | 464.0 | — | c20 |
| value | replace | 40.0 | 565.0 | — | c20 |
| mid_tier | fixed | 11.0 | 559.0 | — | c02 |
| mid_tier | replace | 21.0 | 642.0 | — | c02 |
| random | fixed | 6.0 | 382.0 | — | c02 |
| random | replace | 6.0 | 381.0 | — | c02 |

---

### Episode 19 (post_merge)

**Voted out:** c08 (Contestant 08)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c03 (Contestant 03) | $380,000 | $412,500 | +32,500 (+8.6%) |
| c01 (Contestant 01) | $390,000 | $420,000 | +30,000 (+7.7%) |
| c13 (Contestant 13) | $415,000 | $445,000 | +30,000 (+7.2%) |
| c02 (Contestant 02) | $402,500 | $430,000 | +27,500 (+6.8%) |
| c07 (Contestant 07) | $407,500 | $435,000 | +27,500 (+6.7%) |
| c18 (Contestant 18) | $392,500 | $420,000 | +27,500 (+7.0%) |
| c04 (Contestant 04) | $400,000 | $425,000 | +25,000 (+6.2%) |
| c05 (Contestant 05) | $402,500 | $427,500 | +25,000 (+6.2%) |
| c06 (Contestant 06) | $402,500 | $427,500 | +25,000 (+6.2%) |
| c09 (Contestant 09) | $397,500 | $422,500 | +25,000 (+6.3%) |
| c10 (Contestant 10) | $402,500 | $427,500 | +25,000 (+6.2%) |
| c11 (Contestant 11) | $402,500 | $427,500 | +25,000 (+6.2%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 6.0 | 470.0 | — | c20 |
| value | replace | 12.0 | 577.0 | — | c20 |
| mid_tier | fixed | 15.0 | 574.0 | — | c02 |
| mid_tier | replace | 18.0 | 660.0 | — | c02 |
| random | fixed | 10.0 | 392.0 | — | c02 |
| random | replace | 10.0 | 391.0 | — | c02 |

---

### Episode 20 (post_merge)

**Voted out:** c20 (Contestant 20)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c08 (Contestant 08) | $405,000 | $440,000 | +35,000 (+8.6%) |
| c02 (Contestant 02) | $430,000 | $462,500 | +32,500 (+7.6%) |
| c03 (Contestant 03) | $412,500 | $445,000 | +32,500 (+7.9%) |
| c07 (Contestant 07) | $435,000 | $467,500 | +32,500 (+7.5%) |
| c01 (Contestant 01) | $420,000 | $447,500 | +27,500 (+6.5%) |
| c04 (Contestant 04) | $425,000 | $452,500 | +27,500 (+6.5%) |
| c09 (Contestant 09) | $422,500 | $450,000 | +27,500 (+6.5%) |
| c16 (Contestant 16) | $425,000 | $452,500 | +27,500 (+6.5%) |
| c18 (Contestant 18) | $420,000 | $447,500 | +27,500 (+6.5%) |
| c21 (Contestant 21) | $425,000 | $452,500 | +27,500 (+6.5%) |
| c22 (Contestant 22) | $425,000 | $452,500 | +27,500 (+6.5%) |
| c05 (Contestant 05) | $427,500 | $452,500 | +25,000 (+5.8%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | -48.0 | 422.0 | — | c20 |
| value | replace | -96.0 | 481.0 | — | c20 |
| mid_tier | fixed | 19.0 | 593.0 | — | c02 |
| mid_tier | replace | -5.0 | 655.0 | — | c02 |
| random | fixed | 16.0 | 408.0 | — | c02 |
| random | replace | 16.0 | 407.0 | — | c02 |

---

### Episode 21 (post_merge)

**Voted out:** c07 (Contestant 07)

**Price changes:**

| Contestant | Before | After | Change |
|------------|--------|-------|--------|
| c02 (Contestant 02) | $462,500 | $497,500 | +35,000 (+7.6%) |
| c20 (Contestant 20) | $432,500 | $467,500 | +35,000 (+8.1%) |
| c08 (Contestant 08) | $440,000 | $472,500 | +32,500 (+7.4%) |
| c13 (Contestant 13) | $470,000 | $502,500 | +32,500 (+6.9%) |
| c01 (Contestant 01) | $447,500 | $477,500 | +30,000 (+6.7%) |
| c03 (Contestant 03) | $445,000 | $475,000 | +30,000 (+6.7%) |
| c18 (Contestant 18) | $447,500 | $477,500 | +30,000 (+6.7%) |
| c04 (Contestant 04) | $452,500 | $480,000 | +27,500 (+6.1%) |
| c05 (Contestant 05) | $452,500 | $480,000 | +27,500 (+6.1%) |
| c06 (Contestant 06) | $452,500 | $480,000 | +27,500 (+6.1%) |
| c09 (Contestant 09) | $450,000 | $477,500 | +27,500 (+6.1%) |
| c10 (Contestant 10) | $452,500 | $480,000 | +27,500 (+6.1%) |
| ... | (12 more) | | |

**Fantasy team impact:**

| Team | Style | Episode Pts | Cumulative | Roster Change | Captain |
|------|-------|-------------|-----------|---------------|---------|
| value | fixed | 0.0 | 422.0 | — | — |
| value | replace | 0.0 | 481.0 | — | — |
| mid_tier | fixed | 19.0 | 612.0 | — | c02 |
| mid_tier | replace | 19.0 | 674.0 | — | c02 |
| random | fixed | 16.0 | 424.0 | — | c02 |
| random | replace | 16.0 | 423.0 | — | c02 |

---
